original data source: https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2015

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# read files into pandas
each df has individual needs & requires unique name; not common problem; unable to find solution/alternative to individually reading files into pandas df despite google, slack & mentor

In [3]:
import os
import glob

path = r'C:\Users\katec\Thinkful\data_collections\NHANES_data\2015_2016_data' # use your path
file_names = glob.glob(path + '/*.csv')

In [4]:
file_names

['C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\body_measures_15_16.csv',
 'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\bpq.csv',
 'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\bp_15_16.csv',
 'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\cbc_15_16.csv',
 'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\demo_15_16.csv',
 'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\hdl_15_16.csv',
 'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\HSCRP_I.csv',
 'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\medical_cond.csv',
 'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\std_chem.csv',
 'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\total_dietary_intake_1.csv',
 'C:\\Users\\katec\\Thinkful\\data_collections\\NHANE

In [5]:
body_measures_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\body_measures_15_16.csv')

In [6]:
bp_quest_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\bpq.csv')

In [7]:
bp_measure_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\bp_15_16.csv')

In [8]:
cbc_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\cbc_15_16.csv')

In [9]:
demog_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\demo_15_16.csv')

In [10]:
hdl_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\hdl_15_16.csv')

In [11]:
hscrp_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\HSCRP_I.csv')

In [12]:
med_cond_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\medical_cond.csv')

In [13]:
std_chem_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\std_chem.csv')

In [14]:
tot_intake1_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\total_dietary_intake_1.csv')

In [15]:
tot_intake2_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\total_dietary_intake_2.csv')

In [16]:
tot_chol_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\tot_chol_15_16.csv')

In [17]:
tot_diet_supp_1_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\tot_diet_supplement_1.csv')

In [18]:
tot_diet_supp_2_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\tot_diet_supplement_2.csv')

In [19]:
wt_hx_df = pd.read_csv(r'C:\\Users\\katec\\Thinkful\\data_collections\\NHANES_data\\2015_2016_data\\weight_history.csv')

# attempts to read files & create df with loops, with ordered dictionary
## attempts to concat, merge, use function

each df has individual needs & requires unique name; not common problem; unable to find solution/alternative to individually reading files into pandas df despite google, slack & mentor

see attempts:
https://github.com/kathieking/Thinkful/blob/master/check42_initial_read_concat-Copy2.ipynb

# reduce each dataframe to useful variables

## body measurements

In [20]:
body_measures_df.head()

,SEQN,BMDSTATS,BMXWT,BMIWT,BMXRECUM,BMIRECUM,BMXHEAD,BMIHEAD,BMXHT,BMIHT,BMXBMI,BMDBMIC,BMXLEG,BMILEG,BMXARML,BMIARML,BMXARMC,BMIARMC,BMXWAIST,BMIWAIST,BMXSAD1,BMXSAD2,BMXSAD3,BMXSAD4,BMDAVSAD,BMDSADCM
0,83732.0,1.0,94.8,0.0,0.0,0.0,0.0,0,184.5,0,27.8,0,43.3,0,43.6,0,35.9,0,101.1,0,22.9,22.7,0.0,0.0,22.8,0
1,83733.0,1.0,90.4,0.0,0.0,0.0,0.0,0,171.4,0,30.8,0,38.0,0,40.0,0,33.2,0,107.9,0,27.5,27.1,0.0,0.0,27.3,0
2,83734.0,1.0,83.4,0.0,0.0,0.0,0.0,0,170.1,0,28.8,0,35.6,0,37.0,0,31.0,0,116.5,0,26.7,26.5,0.0,0.0,26.6,0
3,83735.0,1.0,109.8,0.0,0.0,0.0,0.0,0,160.9,0,42.4,0,38.5,0,37.7,0,38.3,0,110.1,0,25.2,25.0,0.0,0.0,25.1,0
4,83736.0,3.0,55.2,0.0,0.0,0.0,0.0,0,164.9,0,20.3,0,37.4,0,36.0,0,27.2,0,80.4,0,0.0,0.0,0.0,0.0,0.0,0


In [21]:
body_measures_df.tail()

,SEQN,BMDSTATS,BMXWT,BMIWT,BMXRECUM,BMIRECUM,BMXHEAD,BMIHEAD,BMXHT,BMIHT,BMXBMI,BMDBMIC,BMXLEG,BMILEG,BMXARML,BMIARML,BMXARMC,BMIARMC,BMXWAIST,BMIWAIST,BMXSAD1,BMXSAD2,BMXSAD3,BMXSAD4,BMDAVSAD,BMDSADCM
9595,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.0,0,165.0,0,21.4,0,38.2,0,33.5,0,26.2,0,72.5,0,16.9,16.9,0.0,0.0,16.9,0
9596,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.0,0,165.0,0,21.4,0,38.2,0,33.5,0,26.2,0,72.5,0,16.9,16.9,0.0,0.0,16.9,0
9597,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.0,0,165.0,0,21.4,0,38.2,0,33.5,0,26.2,0,72.5,0,16.9,16.9,0.0,0.0,16.9,0
9598,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.0,0,165.0,0,21.4,0,38.2,0,33.5,0,26.2,0,72.5,0,16.9,16.9,0.0,0.0,16.9,0
9599,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.0,0,165.0,0,21.4,0,38.2,0,33.5,0,26.2,0,72.5,0,16.9,16.9,0.0,0.0,16.9,0


In [22]:
body_measures_df['SEQN'].nunique()

9545

In [23]:
body_measures_df = body_measures_df.drop(['BMIWT', 'BMXRECUM', 'BMIRECUM', 'BMXHEAD', 
                      'BMIHEAD', 'BMIHT', 'BMXLEG', 'BMILEG', 'BMXARML',
                      'BMIARML', 'BMIWAIST', 'BMXSAD1', 'BMXSAD2',
                      'BMXSAD3', 'BMXSAD4', 'BMDSADCM'], axis=1)
body_measures_df.head()

,SEQN,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMDBMIC,BMXARMC,BMIARMC,BMXWAIST,BMDAVSAD
0,83732.0,1.0,94.8,184.5,27.8,0,35.9,0,101.1,22.8
1,83733.0,1.0,90.4,171.4,30.8,0,33.2,0,107.9,27.3
2,83734.0,1.0,83.4,170.1,28.8,0,31.0,0,116.5,26.6
3,83735.0,1.0,109.8,160.9,42.4,0,38.3,0,110.1,25.1
4,83736.0,3.0,55.2,164.9,20.3,0,27.2,0,80.4,0.0


In [24]:
# no measure = 0, change 0 to NaN
body_measures_df = body_measures_df.replace(0, np.nan)

In [25]:
body_measures_df.isnull().sum()

SEQN           0
BMDSTATS       0
BMXWT         99
BMXHT        775
BMXBMI       788
BMDBMIC     6260
BMXARMC      568
BMIARMC     9179
BMXWAIST    1231
BMDAVSAD    2561
dtype: int64

In [26]:
body_measures_df = body_measures_df.drop(['BMDBMIC', 'BMIARMC'], axis=1)

In [27]:
body_measures_df.head()

,SEQN,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD
0,83732.0,1.0,94.8,184.5,27.8,35.9,101.1,22.8
1,83733.0,1.0,90.4,171.4,30.8,33.2,107.9,27.3
2,83734.0,1.0,83.4,170.1,28.8,31.0,116.5,26.6
3,83735.0,1.0,109.8,160.9,42.4,38.3,110.1,25.1
4,83736.0,3.0,55.2,164.9,20.3,27.2,80.4,NaN


In [28]:
body_measures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9600 entries, 0 to 9599
Data columns (total 8 columns):
SEQN        9600 non-null float64
BMDSTATS    9600 non-null float64
BMXWT       9501 non-null float64
BMXHT       8825 non-null float64
BMXBMI      8812 non-null float64
BMXARMC     9032 non-null float64
BMXWAIST    8369 non-null float64
BMDAVSAD    7039 non-null float64
dtypes: float64(8)
memory usage: 600.1 KB


## blood pressure questions

In [29]:
bp_quest_df.head()

,SEQN,BPQ020,BPQ030,BPD035,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D
0,83732.0,2.0,0.0,0,0,0,2,1,1,2,0
1,83733.0,2.0,0.0,0,0,0,2,2,0,0,0
2,83734.0,1.0,1.0,55,1,1,1,0,1,1,2
3,83735.0,2.0,0.0,0,0,0,2,1,2,2,0
4,83736.0,2.0,0.0,0,0,0,2,1,1,2,0


In [30]:
bp_quest_df.tail()

,SEQN,BPQ020,BPQ030,BPD035,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D
6395,3.687825e-40,3.687825e-40,3.687825e-40,0,0,0,2,1,1,2,0
6396,3.687825e-40,3.687825e-40,3.687825e-40,0,0,0,2,1,1,2,0
6397,3.687825e-40,3.687825e-40,3.687825e-40,0,0,0,2,1,1,2,0
6398,3.687825e-40,3.687825e-40,3.687825e-40,0,0,0,2,1,1,2,0
6399,3.687825e-40,3.687825e-40,3.687825e-40,0,0,0,2,1,1,2,0


In [31]:
bp_quest_df['SEQN'].nunique()

6328

In [32]:
# no measure = 0, change 0 to NaN
bp_quest_df = bp_quest_df.replace(0, np.nan)

In [33]:
bp_quest_df.isnull().sum()

SEQN          0
BPQ020        0
BPQ030     4237
BPD035     4311
BPQ040A    4310
BPQ050A    4549
BPQ080        0
BPQ060     1960
BPQ070     1689
BPQ090D    1689
BPQ100D    4876
dtype: int64

#### can fill missing values for questions using "don't know" value

In [34]:
# drop question "age told have high bp"
bp_quest_df = bp_quest_df.drop(['BPD035'], axis=1)

In [35]:
bp_quest_df.head()

,SEQN,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D
0,83732.0,2.0,NaN,NaN,NaN,2,1.0,1.0,2.0,NaN
1,83733.0,2.0,NaN,NaN,NaN,2,2.0,NaN,NaN,NaN
2,83734.0,1.0,1.0,1.0,1.0,1,NaN,1.0,1.0,2.0
3,83735.0,2.0,NaN,NaN,NaN,2,1.0,2.0,2.0,NaN
4,83736.0,2.0,NaN,NaN,NaN,2,1.0,1.0,2.0,NaN


In [36]:
bp_quest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6400 entries, 0 to 6399
Data columns (total 10 columns):
SEQN       6400 non-null float64
BPQ020     6400 non-null float64
BPQ030     2163 non-null float64
BPQ040A    2090 non-null float64
BPQ050A    1851 non-null float64
BPQ080     6400 non-null int64
BPQ060     4440 non-null float64
BPQ070     4711 non-null float64
BPQ090D    4711 non-null float64
BPQ100D    1524 non-null float64
dtypes: float64(9), int64(1)
memory usage: 500.1 KB


## blood pressure measurements

In [37]:
bp_measure_df.head()

,SEQN,PEASCCT1,BPXCHR,BPAARM,BPACSZ,BPXPLS,BPXPULS,BPXPTY,BPXML1,BPXSY1,BPXDI1,BPAEN1,BPXSY2,BPXDI2,BPAEN2,BPXSY3,BPXDI3,BPAEN3,BPXSY4,BPXDI4,BPAEN4
0,83732.0,0.0,0.0,1.0,4.0,76.0,1,1,150,128,70,2,124,64,2,116,62,2,0,0,0
1,83733.0,0.0,0.0,1.0,4.0,72.0,1,1,170,146,88,2,140,88,2,134,82,2,0,0,0
2,83734.0,0.0,0.0,1.0,4.0,56.0,1,1,160,138,46,2,132,44,2,136,46,2,0,0,0
3,83735.0,0.0,0.0,1.0,5.0,78.0,1,1,150,132,72,2,134,68,2,136,70,2,0,0,0
4,83736.0,0.0,0.0,1.0,3.0,76.0,1,1,130,100,70,2,114,54,2,98,56,2,0,0,0


In [38]:
bp_measure_df.tail()

,SEQN,PEASCCT1,BPXCHR,BPAARM,BPACSZ,BPXPLS,BPXPULS,BPXPTY,BPXML1,BPXSY1,BPXDI1,BPAEN1,BPXSY2,BPXDI2,BPAEN2,BPXSY3,BPXDI3,BPAEN3,BPXSY4,BPXDI4,BPAEN4
9595,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,1,1,150,118,66,2,114,68,2,124,64,2,0,0,0
9596,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,1,1,150,118,66,2,114,68,2,124,64,2,0,0,0
9597,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,1,1,150,118,66,2,114,68,2,124,64,2,0,0,0
9598,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,1,1,150,118,66,2,114,68,2,124,64,2,0,0,0
9599,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,1,1,150,118,66,2,114,68,2,124,64,2,0,0,0


In [39]:
bp_measure_df['SEQN'].nunique()

9545

In [40]:
bp_measure_df = bp_measure_df.drop(['PEASCCT1', 'BPXCHR', 'BPAARM', 
                                    'BPACSZ', 'BPXPLS', 'BPXPULS', 
                                    'BPXPTY', 'BPXML1', 'BPAEN1',
                                    'BPAEN2', 'BPAEN3', 'BPAEN4'], axis=1)
bp_measure_df.head()

,SEQN,BPXSY1,BPXDI1,BPXSY2,BPXDI2,BPXSY3,BPXDI3,BPXSY4,BPXDI4
0,83732.0,128,70,124,64,116,62,0,0
1,83733.0,146,88,140,88,134,82,0,0
2,83734.0,138,46,132,44,136,46,0,0
3,83735.0,132,72,134,68,136,70,0,0
4,83736.0,100,70,114,54,98,56,0,0


In [41]:
# no measure = 0, change 0 to NaN
bp_measure_df = bp_measure_df.replace(0, np.nan)

In [42]:
bp_measure_df.isnull().sum()

SEQN         0
BPXSY1    2399
BPXDI1    2457
BPXSY2    2231
BPXDI2    2297
BPXSY3    2268
BPXDI3    2345
BPXSY4    9276
BPXDI4    9284
dtype: int64

### create average from 4 bp readings, delete individual readings

In [43]:
bp_measure_df['bp_sys_avg'] = bp_measure_df[['BPXSY1','BPXSY2','BPXSY3','BPXSY4']].mean(axis=1)

In [44]:
bp_measure_df['bp_dis_avg'] = bp_measure_df[['BPXDI1', 'BPXDI2', 'BPXDI3', 'BPXDI4']].mean(axis=1)

In [45]:
bp_measure_df.head()

,SEQN,BPXSY1,BPXDI1,BPXSY2,BPXDI2,BPXSY3,BPXDI3,BPXSY4,BPXDI4,bp_sys_avg,bp_dis_avg
0,83732.0,128.0,70.0,124.0,64.0,116.0,62.0,NaN,NaN,122.666667,65.333333
1,83733.0,146.0,88.0,140.0,88.0,134.0,82.0,NaN,NaN,140.000000,86.000000
2,83734.0,138.0,46.0,132.0,44.0,136.0,46.0,NaN,NaN,135.333333,45.333333
3,83735.0,132.0,72.0,134.0,68.0,136.0,70.0,NaN,NaN,134.000000,70.000000
4,83736.0,100.0,70.0,114.0,54.0,98.0,56.0,NaN,NaN,104.000000,60.000000


In [46]:
bp_measure_df = bp_measure_df.drop(['BPXSY1','BPXSY2','BPXSY3','BPXSY4',
                                 'BPXDI1', 'BPXDI2', 'BPXDI3', 'BPXDI4'], axis=1)

In [47]:
bp_measure_df.isnull().sum()

SEQN             0
bp_sys_avg    2181
bp_dis_avg    2209
dtype: int64

In [48]:
bp_measure_df.head()

,SEQN,bp_sys_avg,bp_dis_avg
0,83732.0,122.666667,65.333333
1,83733.0,140.000000,86.000000
2,83734.0,135.333333,45.333333
3,83735.0,134.000000,70.000000
4,83736.0,104.000000,60.000000


In [49]:
bp_measure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9600 entries, 0 to 9599
Data columns (total 3 columns):
SEQN          9600 non-null float64
bp_sys_avg    7419 non-null float64
bp_dis_avg    7391 non-null float64
dtypes: float64(3)
memory usage: 225.1 KB


## complete blood count

In [50]:
cbc_df.head()

,SEQN,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBDBANO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXMCH,LBXRDW,LBXPLTSI,LBXMPSI
0,83732.0,9.8,23.9,8.2,63.5,4.0,0.5,2.3,0.8,6.2,0.4,0.0,4.93,15.2,44.7,90.8,30.8,0,13.9,181,8.3
1,83733.0,7.3,31.3,9.7,54.8,2.6,1.8,2.3,0.7,4.0,0.2,0.1,4.89,17.5,49.7,101.8,35.8,0,13.4,170,9.6
2,83734.0,4.4,29.9,9.6,55.8,3.9,0.9,1.3,0.4,2.5,0.2,0.0,4.18,12.4,37.9,90.8,29.6,0,14.7,223,9.0
3,83735.0,6.1,17.1,10.3,68.7,3.1,0.9,1.0,0.6,4.2,0.2,0.1,4.54,12.8,40.1,88.3,28.2,0,13.1,280,9.1
4,83736.0,4.2,47.1,7.8,44.8,0.2,0.2,2.0,0.3,1.9,0.0,0.0,4.16,12.1,36.5,87.8,29.1,0,12.3,275,7.7


In [51]:
cbc_df.tail()

,SEQN,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBDBANO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXMCH,LBXRDW,LBXPLTSI,LBXMPSI
9195,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,0.0,0.0,4.58,12.8,39.4,86.3,28.0,0,13.8,257,8.7
9196,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,0.0,0.0,4.58,12.8,39.4,86.3,28.0,0,13.8,257,8.7
9197,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,0.0,0.0,4.58,12.8,39.4,86.3,28.0,0,13.8,257,8.7
9198,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,0.0,0.0,4.58,12.8,39.4,86.3,28.0,0,13.8,257,8.7
9199,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,0.0,0.0,4.58,12.8,39.4,86.3,28.0,0,13.8,257,8.7


In [52]:
cbc_df['SEQN'].nunique()

9166

In [53]:
cbc_df = cbc_df.replace(0, np.nan)

In [54]:
cbc_df.isnull().sum()

SEQN           0
LBXWBCSI    1048
LBXLYPCT    1049
LBXMOPCT    1049
LBXNEPCT    1049
LBXEOPCT    1066
LBXBAPCT    1049
LBDLYMNO    1049
LBDMONO     1049
LBDNENO     1049
LBDEONO     1495
LBDBANO     4940
LBXRBCSI    1048
LBXHGB      1048
LBXHCT      1048
LBXMCVSI    1048
LBXMCHSI    1048
LBXMCH      9200
LBXRDW      1048
LBXPLTSI    1048
LBXMPSI     1048
dtype: int64

In [55]:
#drop LBXMCH
cbc_df = cbc_df.drop(['LBXMCH', 'LBDBANO'], axis=1)

In [56]:
cbc_df.head()

,SEQN,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI
0,83732.0,9.8,23.9,8.2,63.5,4.0,0.5,2.3,0.8,6.2,0.4,4.93,15.2,44.7,90.8,30.8,13.9,181.0,8.3
1,83733.0,7.3,31.3,9.7,54.8,2.6,1.8,2.3,0.7,4.0,0.2,4.89,17.5,49.7,101.8,35.8,13.4,170.0,9.6
2,83734.0,4.4,29.9,9.6,55.8,3.9,0.9,1.3,0.4,2.5,0.2,4.18,12.4,37.9,90.8,29.6,14.7,223.0,9.0
3,83735.0,6.1,17.1,10.3,68.7,3.1,0.9,1.0,0.6,4.2,0.2,4.54,12.8,40.1,88.3,28.2,13.1,280.0,9.1
4,83736.0,4.2,47.1,7.8,44.8,0.2,0.2,2.0,0.3,1.9,NaN,4.16,12.1,36.5,87.8,29.1,12.3,275.0,7.7


In [57]:
cbc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9200 entries, 0 to 9199
Data columns (total 19 columns):
SEQN        9200 non-null float64
LBXWBCSI    8152 non-null float64
LBXLYPCT    8151 non-null float64
LBXMOPCT    8151 non-null float64
LBXNEPCT    8151 non-null float64
LBXEOPCT    8134 non-null float64
LBXBAPCT    8151 non-null float64
LBDLYMNO    8151 non-null float64
LBDMONO     8151 non-null float64
LBDNENO     8151 non-null float64
LBDEONO     7705 non-null float64
LBXRBCSI    8152 non-null float64
LBXHGB      8152 non-null float64
LBXHCT      8152 non-null float64
LBXMCVSI    8152 non-null float64
LBXMCHSI    8152 non-null float64
LBXRDW      8152 non-null float64
LBXPLTSI    8152 non-null float64
LBXMPSI     8152 non-null float64
dtypes: float64(19)
memory usage: 1.3 MB


## demographics

In [58]:
demog_df.head()

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMQADFC,DMDBORN4,DMDCITZN,DMDYRSUS,DMDEDUC3,DMDEDUC2,DMDMARTL,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,DMDHRGND,DMDHRAGE,DMDHRBR4,DMDHREDU,DMDHRMAR,DMDHSEDU,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
0,83732.0,9.0,2.0,1,62,0,3,3,1,0,2,0,1,1,0,0,5,1,0,1,2,2,1,2,2,1,2,2,1,2,2,0,0,1,1,62,1,5,1,3,134671.370419,135629.507405,1,125,10,10,4.39
1,83733.0,9.0,2.0,1,53,0,3,3,1,0,2,0,2,2,7,0,3,3,0,1,2,2,1,2,2,1,2,2,1,1,1,0,0,0,1,53,2,3,3,0,24328.560239,25282.425927,1,125,4,4,1.32
2,83734.0,9.0,2.0,1,78,0,3,3,2,0,1,2,1,1,0,0,3,1,0,1,2,2,1,2,2,1,2,2,0,2,2,0,0,2,2,79,1,3,1,3,12400.008522,12575.838818,1,131,5,5,1.51
3,83735.0,9.0,2.0,2,56,0,3,3,2,0,2,0,1,1,0,0,5,6,0,1,2,2,1,2,2,1,2,2,1,1,1,0,0,0,2,56,1,5,6,0,102717.995647,102078.634508,1,131,10,10,5.00
4,83736.0,9.0,2.0,2,42,0,4,4,2,0,2,0,1,1,0,0,4,3,1,1,2,2,1,2,2,1,2,2,1,5,5,0,2,0,2,42,1,4,3,0,17627.674984,18234.736219,2,126,7,7,1.23


In [59]:
demog_df.tail()

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMQADFC,DMDBORN4,DMDCITZN,DMDYRSUS,DMDEDUC3,DMDEDUC2,DMDMARTL,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,DMDHRGND,DMDHRAGE,DMDHRBR4,DMDHREDU,DMDHRMAR,DMDHSEDU,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
9995,3.687825e-40,3.687825e-40,3.687825e-40,2,24,0,3,3,2,0,2,0,1,1,0,0,5,5,2,1,2,2,1,2,2,1,2,2,1,3,1,0,0,0,2,22,1,5,5,0,107361.906644,105080.445194,2,119,10,7,3.54
9996,3.687825e-40,3.687825e-40,3.687825e-40,2,24,0,3,3,2,0,2,0,1,1,0,0,5,5,2,1,2,2,1,2,2,1,2,2,1,3,1,0,0,0,2,22,1,5,5,0,107361.906644,105080.445194,2,119,10,7,3.54
9997,3.687825e-40,3.687825e-40,3.687825e-40,2,24,0,3,3,2,0,2,0,1,1,0,0,5,5,2,1,2,2,1,2,2,1,2,2,1,3,1,0,0,0,2,22,1,5,5,0,107361.906644,105080.445194,2,119,10,7,3.54
9998,3.687825e-40,3.687825e-40,3.687825e-40,2,24,0,3,3,2,0,2,0,1,1,0,0,5,5,2,1,2,2,1,2,2,1,2,2,1,3,1,0,0,0,2,22,1,5,5,0,107361.906644,105080.445194,2,119,10,7,3.54
9999,3.687825e-40,3.687825e-40,3.687825e-40,2,24,0,3,3,2,0,2,0,1,1,0,0,5,5,2,1,2,2,1,2,2,1,2,2,1,3,1,0,0,0,2,22,1,5,5,0,107361.906644,105080.445194,2,119,10,7,3.54


In [60]:
demog_df['SEQN'].nunique()

9972

In [61]:
demog_df = demog_df.drop(['SDDSRVYR', 'RIDSTATR', 'RIDAGEMN', 'RIDRETH1',
                          'RIDRETH3', 'RIDEXMON', 'DMQMILIZ', 'DMQADFC', 
                          'DMDBORN4', 'DMDCITZN', 'DMDYRSUS', 'DMDEDUC3', 
                          'DMDEDUC2', 'DMDMARTL', 'RIDEXPRG', 'SIALANG', 
                          'SIAPROXY', 'SIAINTRP', 'FIALANG', 'FIAPROXY',
                          'FIAINTRP', 'MIALANG', 'MIAPROXY', 'MIAINTRP',
                          'AIALANGA', 'DMDFMSIZ', 
                          'DMDHHSIZ', 'DMDHHSZA', 'DMDHHSZB', 'DMDHHSZE', 
                          'DMDHRGND', 'DMDHRAGE', 'DMDHRBR4', 'DMDHREDU', 
                          'DMDHRMAR', 'DMDHSEDU', 'SDMVPSU', 'SDMVSTRA', 
                          'INDHHIN2', 'INDFMIN2'], axis=1)
demog_df.head()

,SEQN,RIAGENDR,RIDAGEYR,RIDEXAGM,WTINT2YR,WTMEC2YR,INDFMPIR
0,83732.0,1,62,0,134671.370419,135629.507405,4.39
1,83733.0,1,53,0,24328.560239,25282.425927,1.32
2,83734.0,1,78,0,12400.008522,12575.838818,1.51
3,83735.0,2,56,0,102717.995647,102078.634508,5.00
4,83736.0,2,42,0,17627.674984,18234.736219,1.23


In [62]:
demog_df = demog_df.drop(['RIDEXAGM'], axis = 1)

In [63]:
demog_df = demog_df.replace(0, np.nan)

In [64]:
demog_df.isnull().sum()

SEQN           0
RIAGENDR       0
RIDAGEYR     396
WTINT2YR       0
WTMEC2YR     427
INDFMPIR    1144
dtype: int64

In [65]:
demog_df.head()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR
0,83732.0,1,62.0,134671.370419,135629.507405,4.39
1,83733.0,1,53.0,24328.560239,25282.425927,1.32
2,83734.0,1,78.0,12400.008522,12575.838818,1.51
3,83735.0,2,56.0,102717.995647,102078.634508,5.00
4,83736.0,2,42.0,17627.674984,18234.736219,1.23


In [66]:
demog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
SEQN        10000 non-null float64
RIAGENDR    10000 non-null int64
RIDAGEYR    9604 non-null float64
WTINT2YR    10000 non-null float64
WTMEC2YR    9573 non-null float64
INDFMPIR    8856 non-null float64
dtypes: float64(5), int64(1)
memory usage: 468.8 KB


## hdl cholesterol

In [67]:
hdl_df.head()

,SEQN,LBDHDD,LBDHDDSI
0,83732.0,46.0,1.19
1,83733.0,63.0,1.63
2,83734.0,30.0,0.78
3,83735.0,61.0,1.58
4,83736.0,53.0,1.37


In [68]:
hdl_df.tail()

,SEQN,LBDHDD,LBDHDDSI
8095,3.687825e-40,3.687825e-40,3.687825e-40
8096,3.687825e-40,3.687825e-40,3.687825e-40
8097,3.687825e-40,3.687825e-40,3.687825e-40
8098,3.687825e-40,3.687825e-40,3.687825e-40
8099,3.687825e-40,3.687825e-40,3.687825e-40


In [69]:
hdl_df['SEQN'].nunique()

8022

In [70]:
hdl_df = hdl_df.drop(['LBDHDDSI'], axis=1)
hdl_df.head()

,SEQN,LBDHDD
0,83732.0,46.0
1,83733.0,63.0
2,83734.0,30.0
3,83735.0,61.0
4,83736.0,53.0


In [71]:
# no measure = 0, change 0 to NaN
hdl_df = hdl_df.replace(0, np.nan)

In [72]:
hdl_df.isnull().sum()

SEQN        0
LBDHDD    765
dtype: int64

In [73]:
hdl_df.head()

,SEQN,LBDHDD
0,83732.0,46.0
1,83733.0,63.0
2,83734.0,30.0
3,83735.0,61.0
4,83736.0,53.0


In [74]:
hdl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8100 entries, 0 to 8099
Data columns (total 2 columns):
SEQN      8100 non-null float64
LBDHDD    7335 non-null float64
dtypes: float64(2)
memory usage: 126.6 KB


## high sensitivity C-reactive protein

In [75]:
hscrp_df.head()

,SEQN,LBXHSCRP,LBDHRPLC
0,83732.0,0.6,0.0
1,83733.0,1.4,0.0
2,83734.0,0.6,0.0
3,83735.0,9.0,0.0
4,83736.0,0.5,0.0


In [76]:
hscrp_df.tail()

,SEQN,LBXHSCRP,LBDHRPLC
9195,3.687825e-40,3.687825e-40,3.687825e-40
9196,3.687825e-40,3.687825e-40,3.687825e-40
9197,3.687825e-40,3.687825e-40,3.687825e-40
9198,3.687825e-40,3.687825e-40,3.687825e-40
9199,3.687825e-40,3.687825e-40,3.687825e-40


In [77]:
hscrp_df['SEQN'].nunique()

9166

In [78]:
hscrp_df = hscrp_df.drop(['LBDHRPLC'], axis=1)
hscrp_df.head()

,SEQN,LBXHSCRP
0,83732.0,0.6
1,83733.0,1.4
2,83734.0,0.6
3,83735.0,9.0
4,83736.0,0.5


In [79]:
# no measure = 0, change 0 to NaN
hscrp_df = hscrp_df.replace(0, np.nan)

In [80]:
hscrp_df.isnull().sum()

SEQN           0
LBXHSCRP    1298
dtype: int64

In [81]:
hscrp_df.head()

,SEQN,LBXHSCRP
0,83732.0,0.6
1,83733.0,1.4
2,83734.0,0.6
3,83735.0,9.0
4,83736.0,0.5


In [82]:
hscrp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9200 entries, 0 to 9199
Data columns (total 2 columns):
SEQN        9200 non-null float64
LBXHSCRP    7902 non-null float64
dtypes: float64(2)
memory usage: 143.8 KB


## standard chemistry

In [83]:
std_chem_df.head()

,SEQN,LBXSAL,LBDSALSI,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBDSBUSI,LBXSC3SI,LBXSCA,LBDSCASI,LBXSCH,LBDSCHSI,LBXSCK,LBXSCLSI,LBXSCR,LBDSCRSI,LBXSGB,LBDSGBSI,LBXSGL,LBDSGLSI,LBXSGTSI,LBXSIR,LBDSIRSI,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBDSTBSI,LBXSTP,LBDSTPSI,LBXSTR,LBDSTRSI,LBXSUA,LBDSUASI
0,83732.0,4.6,46.0,52.0,21.0,25.0,13.0,4.64,25,9.8,2.450,172,4.448,203,100,0.90,79.56,2.9,29,94,5.22,22,70,12.5,4.30,110,136,272,4.7,1.518,0.5,8.55,7.5,75,158,1.784,4.2,249.8
1,83733.0,4.5,45.0,47.0,31.0,35.0,10.0,3.57,27,9.8,2.450,267,6.905,91,103,1.05,92.82,2.9,29,94,5.22,75,161,28.8,4.55,108,140,278,4.4,1.421,0.6,10.26,7.4,74,170,1.919,7.0,416.4
2,83734.0,4.5,45.0,46.0,30.0,29.0,26.0,9.28,24,9.7,2.425,236,6.103,160,106,1.12,99.01,2.8,28,103,5.72,11,72,12.9,4.70,151,140,284,3.6,1.162,0.5,8.55,7.3,73,299,3.376,7.3,434.2
3,83735.0,3.8,38.0,65.0,23.0,26.0,13.0,4.64,24,8.9,2.225,179,4.629,66,106,0.96,84.86,2.3,23,63,3.50,19,43,7.7,4.08,172,141,279,3.8,1.227,0.3,5.13,6.1,61,93,1.050,5.4,321.2
4,83736.0,4.3,43.0,46.0,20.0,13.0,12.0,4.28,24,9.3,2.325,202,5.224,67,99,0.64,56.58,3.4,34,83,4.61,23,44,7.9,3.50,93,136,270,3.2,1.033,0.3,5.13,7.7,77,52,0.587,3.3,196.3


In [84]:
std_chem_df.tail()

,SEQN,LBXSAL,LBDSALSI,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBDSBUSI,LBXSC3SI,LBXSCA,LBDSCASI,LBXSCH,LBDSCHSI,LBXSCK,LBXSCLSI,LBXSCR,LBDSCRSI,LBXSGB,LBDSGBSI,LBXSGL,LBDSGLSI,LBXSGTSI,LBXSIR,LBDSIRSI,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBDSTBSI,LBXSTP,LBDSTPSI,LBXSTR,LBDSTRSI,LBXSUA,LBDSUASI
6795,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25,9.0,2.25,163,4.215,78,102,0.66,58.34,2.9,29,101,5.61,11,156,27.9,3.84,93,138,275,2.8,0.904,0.5,8.55,6.8,68,79,0.892,4.2,249.8
6796,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25,9.0,2.25,163,4.215,78,102,0.66,58.34,2.9,29,101,5.61,11,156,27.9,3.84,93,138,275,2.8,0.904,0.5,8.55,6.8,68,79,0.892,4.2,249.8
6797,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25,9.0,2.25,163,4.215,78,102,0.66,58.34,2.9,29,101,5.61,11,156,27.9,3.84,93,138,275,2.8,0.904,0.5,8.55,6.8,68,79,0.892,4.2,249.8
6798,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25,9.0,2.25,163,4.215,78,102,0.66,58.34,2.9,29,101,5.61,11,156,27.9,3.84,93,138,275,2.8,0.904,0.5,8.55,6.8,68,79,0.892,4.2,249.8
6799,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25,9.0,2.25,163,4.215,78,102,0.66,58.34,2.9,29,101,5.61,11,156,27.9,3.84,93,138,275,2.8,0.904,0.5,8.55,6.8,68,79,0.892,4.2,249.8


In [85]:
std_chem_df['SEQN'].nunique()

6745

In [86]:
std_chem_df = std_chem_df.drop(['LBDSALSI','LBDSBUSI','LBDSCASI','LBDSCHSI',
                                'LBDSCRSI', 'LBXSGB', 'LBDSGBSI', 'LBDSGLSI', 
                                'LBXSGTSI', 'LBDSIRSI', 'LBDSTBSI', 'LBDSTPSI', 
                                'LBDSTRSI', 'LBDSUASI'], axis=1)
std_chem_df.head()

,SEQN,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA
0,83732.0,4.6,52.0,21.0,25.0,13.0,25,9.8,172,203,100,0.90,94,70,4.30,110,136,272,4.7,1.518,0.5,7.5,158,4.2
1,83733.0,4.5,47.0,31.0,35.0,10.0,27,9.8,267,91,103,1.05,94,161,4.55,108,140,278,4.4,1.421,0.6,7.4,170,7.0
2,83734.0,4.5,46.0,30.0,29.0,26.0,24,9.7,236,160,106,1.12,103,72,4.70,151,140,284,3.6,1.162,0.5,7.3,299,7.3
3,83735.0,3.8,65.0,23.0,26.0,13.0,24,8.9,179,66,106,0.96,63,43,4.08,172,141,279,3.8,1.227,0.3,6.1,93,5.4
4,83736.0,4.3,46.0,20.0,13.0,12.0,24,9.3,202,67,99,0.64,83,44,3.50,93,136,270,3.2,1.033,0.3,7.7,52,3.3


In [87]:
std_chem_df = std_chem_df.replace(0, np.nan)

In [88]:
std_chem_df.isnull().sum()

SEQN          0
LBXSAL      488
LBXSAPSI    489
LBXSASSI    488
LBXSATSI    488
LBXSBU      489
LBXSC3SI    487
LBXSCA      487
LBXSCH      490
LBXSCK      496
LBXSCLSI    487
LBXSCR      489
LBXSGL      487
LBXSIR      491
LBXSKSI     487
LBXSLDSI    488
LBXSNASI    487
LBXSOSSI    490
LBXSPH      488
LBDSPHSI    488
LBXSTB      517
LBXSTP      491
LBXSTR      490
LBXSUA      490
dtype: int64

In [89]:
std_chem_df.head()

,SEQN,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA
0,83732.0,4.6,52.0,21.0,25.0,13.0,25.0,9.8,172.0,203.0,100.0,0.90,94.0,70.0,4.30,110.0,136.0,272.0,4.7,1.518,0.5,7.5,158.0,4.2
1,83733.0,4.5,47.0,31.0,35.0,10.0,27.0,9.8,267.0,91.0,103.0,1.05,94.0,161.0,4.55,108.0,140.0,278.0,4.4,1.421,0.6,7.4,170.0,7.0
2,83734.0,4.5,46.0,30.0,29.0,26.0,24.0,9.7,236.0,160.0,106.0,1.12,103.0,72.0,4.70,151.0,140.0,284.0,3.6,1.162,0.5,7.3,299.0,7.3
3,83735.0,3.8,65.0,23.0,26.0,13.0,24.0,8.9,179.0,66.0,106.0,0.96,63.0,43.0,4.08,172.0,141.0,279.0,3.8,1.227,0.3,6.1,93.0,5.4
4,83736.0,4.3,46.0,20.0,13.0,12.0,24.0,9.3,202.0,67.0,99.0,0.64,83.0,44.0,3.50,93.0,136.0,270.0,3.2,1.033,0.3,7.7,52.0,3.3


In [90]:
std_chem_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6800 entries, 0 to 6799
Data columns (total 24 columns):
SEQN        6800 non-null float64
LBXSAL      6312 non-null float64
LBXSAPSI    6311 non-null float64
LBXSASSI    6312 non-null float64
LBXSATSI    6312 non-null float64
LBXSBU      6311 non-null float64
LBXSC3SI    6313 non-null float64
LBXSCA      6313 non-null float64
LBXSCH      6310 non-null float64
LBXSCK      6304 non-null float64
LBXSCLSI    6313 non-null float64
LBXSCR      6311 non-null float64
LBXSGL      6313 non-null float64
LBXSIR      6309 non-null float64
LBXSKSI     6313 non-null float64
LBXSLDSI    6312 non-null float64
LBXSNASI    6313 non-null float64
LBXSOSSI    6310 non-null float64
LBXSPH      6312 non-null float64
LBDSPHSI    6312 non-null float64
LBXSTB      6283 non-null float64
LBXSTP      6309 non-null float64
LBXSTR      6310 non-null float64
LBXSUA      6310 non-null float64
dtypes: float64(24)
memory usage: 1.2 MB


## medical condition

In [91]:
med_cond_df.head()

,SEQN,MCQ010,MCQ025,MCQ035,MCQ040,MCQ050,AGQ030,MCQ053,MCQ080,MCQ092,MCD093,MCQ149,MCQ151,MCQ160A,MCQ180A,MCQ195,MCQ160N,MCQ180N,MCQ160B,MCQ180B,MCQ160C,MCQ180C,MCQ160D,MCQ180D,MCQ160E,MCQ180E,MCQ160F,MCQ180F,MCQ160G,MCQ180G,MCQ160M,MCQ170M,MCQ180M,MCQ160K,MCQ170K,MCQ180K,MCQ160L,MCQ170L,MCQ180L,MCQ160O,MCQ203,MCQ206,MCQ220,MCQ230A,MCQ230B,MCQ230C,MCQ230D,MCQ240A,MCQ240AA,MCQ240B,MCQ240BB,MCQ240C,MCQ240CC,MCQ240D,MCQ240DD,MCQ240DK,MCQ240E,MCQ240F,MCQ240G,MCQ240H,MCQ240I,MCQ240J,MCQ240K,MCQ240L,MCQ240M,MCQ240N,MCQ240O,MCQ240P,MCQ240Q,MCQ240R,MCQ240S,MCQ240T,MCQ240U,MCQ240V,MCQ240W,MCQ240X,MCQ240Y,MCQ240Z,MCQ300A,MCQ300B,MCQ300C,MCQ365A,MCQ365B,MCQ365C,MCQ365D,MCQ370A,MCQ370B,MCQ370C,MCQ370D,OSQ230
0,83732,2,0,0,0,0,0,2,1,2,0,0,0,1,40,1,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,0,2,0,0,2,0,0,2,2,0,1,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,58,0,0,0,0,0,0,0,0,0,0,2,2,1,2,1,2,2,1,1,2,2,1
1,83733,2,0,0,0,0,0,2,2,2,0,0,0,2,0,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,0,2,0,0,2,0,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,1,2,2,2,2,2,2,2,2,2
2,83734,1,60,1,2,2,1,2,1,2,0,0,0,1,55,4,2,0,2,0,2,0,2,0,1,58,2,0,1,59,1,2,39,2,0,0,1,1,11,1,1,11,1,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,57,0,0,2,1,2,2,1,1,1,1,2,2,1,2
3,83735,2,0,0,0,0,0,2,1,2,0,0,0,2,0,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,0,2,0,0,2,0,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,1,1,2,1,1,2,1,1,2
4,83736,1,10,2,0,0,0,2,2,2,0,0,0,1,10,9,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,0,2,0,0,2,0,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,9,2,2,2,2,2,2,2,2,2


In [92]:
med_cond_df.tail()

,SEQN,MCQ010,MCQ025,MCQ035,MCQ040,MCQ050,AGQ030,MCQ053,MCQ080,MCQ092,MCD093,MCQ149,MCQ151,MCQ160A,MCQ180A,MCQ195,MCQ160N,MCQ180N,MCQ160B,MCQ180B,MCQ160C,MCQ180C,MCQ160D,MCQ180D,MCQ160E,MCQ180E,MCQ160F,MCQ180F,MCQ160G,MCQ180G,MCQ160M,MCQ170M,MCQ180M,MCQ160K,MCQ170K,MCQ180K,MCQ160L,MCQ170L,MCQ180L,MCQ160O,MCQ203,MCQ206,MCQ220,MCQ230A,MCQ230B,MCQ230C,MCQ230D,MCQ240A,MCQ240AA,MCQ240B,MCQ240BB,MCQ240C,MCQ240CC,MCQ240D,MCQ240DD,MCQ240DK,MCQ240E,MCQ240F,MCQ240G,MCQ240H,MCQ240I,MCQ240J,MCQ240K,MCQ240L,MCQ240M,MCQ240N,MCQ240O,MCQ240P,MCQ240Q,MCQ240R,MCQ240S,MCQ240T,MCQ240U,MCQ240V,MCQ240W,MCQ240X,MCQ240Y,MCQ240Z,MCQ300A,MCQ300B,MCQ300C,MCQ365A,MCQ365B,MCQ365C,MCQ365D,MCQ370A,MCQ370B,MCQ370C,MCQ370D,OSQ230
9595,93702,2,0,0,0,0,0,2,2,2,0,0,0,2,0,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,0,2,0,0,2,0,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,1,1,1,1,0
9596,93702,2,0,0,0,0,0,2,2,2,0,0,0,2,0,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,0,2,0,0,2,0,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,1,1,1,1,0
9597,93702,2,0,0,0,0,0,2,2,2,0,0,0,2,0,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,0,2,0,0,2,0,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,1,1,1,1,0
9598,93702,2,0,0,0,0,0,2,2,2,0,0,0,2,0,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,0,2,0,0,2,0,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,1,1,1,1,0
9599,93702,2,0,0,0,0,0,2,2,2,0,0,0,2,0,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,0,2,0,0,2,0,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,1,1,1,1,0


In [93]:
med_cond_df['SEQN'].nunique()

9575

In [94]:
med_cond_df = med_cond_df.drop(['MCQ010','MCQ025','MCQ035','MCQ040', 'MCQ050',
                                'AGQ030', 'MCQ053', 'MCQ080', 'MCQ092', 
                                'MCD093', 'MCQ149', 'MCQ151', 'MCQ160A', 
                                'MCQ180A', 'MCQ195', 'MCQ160N', 'MCQ180N', 
                                'MCQ160F', 'MCQ180F', 'MCQ160G', 'MCQ180G', 
                                'MCQ160M', 'MCQ170M', 'MCQ180M', 'MCQ160K', 
                                'MCQ170K', 'MCQ160L', 'MCQ170L', 'MCQ180L', 
                                'MCQ160O', 'MCQ203', 'MCQ206', 'MCQ220', 
                                'MCQ230A', 'MCQ230B', 'MCQ230C', 'MCQ230D', 
                                'MCQ240A', 'MCQ240AA', 'MCQ240B', 'MCQ240BB', 
                                'MCQ240C', 'MCQ240CC', 'MCQ240D', 'MCQ240DD', 
                                'MCQ240DK', 'MCQ240E', 'MCQ240F', 'MCQ240G', 
                                'MCQ240H', 'MCQ240I', 'MCQ240J', 'MCQ240K', 
                                'MCQ240L', 'MCQ240M', 'MCQ240N', 'MCQ240O', 
                                'MCQ240P', 'MCQ240Q', 'MCQ240R', 'MCQ240S', 
                                'MCQ240T', 'MCQ240U', 'MCQ240V', 'MCQ240W', 
                                'MCQ240X', 'MCQ240Y', 'MCQ240Z', 'MCQ300B', 
                                'MCQ300C', 'MCQ365A', 'MCQ365B', 'MCQ370B', 
                                'OSQ230'], axis=1)
med_cond_df.head()

,SEQN,MCQ160B,MCQ180B,MCQ160C,MCQ180C,MCQ160D,MCQ180D,MCQ160E,MCQ180E,MCQ180K,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D
0,83732,2,0,2,0,2,0,2,0,0,2,2,2,1,2,2
1,83733,2,0,2,0,2,0,2,0,0,2,2,2,2,2,2
2,83734,2,0,2,0,2,0,1,58,0,2,1,1,1,2,1
3,83735,2,0,2,0,2,0,2,0,0,9,2,1,1,1,1
4,83736,2,0,2,0,2,0,2,0,0,2,2,2,2,2,2


In [95]:
med_cond_df = med_cond_df.replace(0, np.nan)

In [96]:
med_cond_df.isnull().sum()

SEQN          0
MCQ160B    3856
MCQ180B    9387
MCQ160C    3856
MCQ180C    9356
MCQ160D    3856
MCQ180D    9467
MCQ160E    3856
MCQ180E    9349
MCQ180K    9297
MCQ300A    3856
MCQ365C    3248
MCQ365D    3248
MCQ370A    3248
MCQ370C    3248
MCQ370D    3248
dtype: int64

In [97]:
med_cond_df = med_cond_df.drop(['MCQ180B', 'MCQ180C', 'MCQ180D', 'MCQ180E', 'MCQ180K'], axis=1)

In [98]:
med_cond_df['SEQN'] = med_cond_df['SEQN'].astype(float)

In [99]:
med_cond_df.head()

,SEQN,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D
0,83732.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0
1,83733.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,83734.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0
3,83735.0,2.0,2.0,2.0,2.0,9.0,2.0,1.0,1.0,1.0,1.0
4,83736.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [100]:
med_cond_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9600 entries, 0 to 9599
Data columns (total 11 columns):
SEQN       9600 non-null float64
MCQ160B    5744 non-null float64
MCQ160C    5744 non-null float64
MCQ160D    5744 non-null float64
MCQ160E    5744 non-null float64
MCQ300A    5744 non-null float64
MCQ365C    6352 non-null float64
MCQ365D    6352 non-null float64
MCQ370A    6352 non-null float64
MCQ370C    6352 non-null float64
MCQ370D    6352 non-null float64
dtypes: float64(11)
memory usage: 825.1 KB


## total dietary intake 1

In [101]:
tot_intake1_df.head()

,SEQN,WTDRD1,WTDR2D,DR1DRSTZ,DR1EXMER,DRABF,DRDINT,DR1DBIH,DR1DAY,DR1LANG,DR1MRESP,DR1HELP,DBQ095Z,DBD100,DRQSPREP,DR1STY,DR1SKY,DRQSDIET,DRQSDT1,DRQSDT2,DRQSDT3,DRQSDT4,DRQSDT5,DRQSDT6,DRQSDT7,DRQSDT8,DRQSDT9,DRQSDT10,DRQSDT11,DRQSDT12,DRQSDT91,DR1TNUMF,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TATOA,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TB12A,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TTHEO,DR1TALCO,DR1TMOIS,DR1TS040,DR1TS060,DR1TS080,DR1TS100,DR1TS120,DR1TS140,DR1TS160,DR1TS180,DR1TM161,DR1TM181,DR1TM201,DR1TM221,DR1TP182,DR1TP183,DR1TP184,DR1TP204,DR1TP205,DR1TP225,DR1TP226,DR1_300,DR1_320Z,DR1_330Z,DR1BWATZ,DR1TWS,DRD340,DRD350A,DRD350AQ,DRD350B,DRD350BQ,DRD350C,DRD350CQ,DRD350D,DRD350DQ,DRD350E,DRD350EQ,DRD350F,DRD350FQ,DRD350G,DRD350GQ,DRD350H,DRD350HQ,DRD350I,DRD350IQ,DRD350J,DRD350JQ,DRD350K,DRD360,DRD370A,DRD370AQ,DRD370B,DRD370BQ,DRD370C,DRD370CQ,DRD370D,DRD370DQ,DRD370E,DRD370EQ,DRD370F,DRD370FQ,DRD370G,DRD370GQ,DRD370H,DRD370HQ,DRD370I,DRD370IQ,DRD370J,DRD370JQ,DRD370K,DRD370KQ,DRD370L,DRD370LQ,DRD370M,DRD370MQ,DRD370N,DRD370NQ,DRD370O,DRD370OQ,DRD370P,DRD370PQ,DRD370Q,DRD370QQ,DRD370R,DRD370RQ,DRD370S,DRD370SQ,DRD370T,DRD370TQ,DRD370U,DRD370UQ,DRD370V
0,83732.0,92670.699919,69945.934107,1.0,71.0,2.0,2.0,-1.0,1,1,1,12,1,3,3,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,17,1781,76.03,193.29,42.31,23.6,79.24,23.430,31.897,18.528,138,10.59,0.0,230,307,336,624,282,1206,1120,2.344,1.949,20.401,2.279,450,163,286,564,290.7,1.89,0.0,110.5,2.6,139.0,623,1052,255,16.01,8.73,1.095,5298,2641,113.6,360,0,0.0,3028.29,0.013,0.010,0.053,0.092,0.365,1.271,13.199,7.334,1.585,28.937,0.332,0.276,16.024,2.294,0.004,0.110,0.004,0.017,0.002,2,960.0,120.0,840.0,2,1,2,0,2,0,2,0,2,0,2,0,2,0,2,0,1,1,2,0,2,0,2,1,1,1,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,1,1,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2
1,83733.0,16454.105994,13190.268870,1.0,61.0,2.0,2.0,17.0,7,1,1,12,4,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2964,62.36,356.85,180.84,7.3,77.91,25.722,19.098,19.216,407,4.19,0.0,352,453,405,966,20,0,1279,1.859,2.478,29.444,1.823,478,112,365,556,672.7,2.92,0.0,30.2,4.6,87.6,594,1414,262,11.01,4.82,0.571,3164,2249,89.9,192,0,89.3,4024.25,0.563,0.441,0.275,0.764,0.863,2.688,13.234,6.144,0.693,17.837,0.155,0.000,16.712,2.140,0.000,0.206,0.005,0.026,0.036,2,507.0,0.0,507.0,4,1,2,0,2,0,2,0,2,0,2,0,2,0,2,0,1,2,2,0,2,0,2,1,2,0,2,0,2,0,1,4,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,1,4,2,0,2
2,83734.0,6529.619788,4928.422423,1.0,14.0,2.0,2.0,6.0,7,1,1,12,1,3,4,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,14,2482,116.50,304.53,62.87,35.1,91.97,23.122,28.666,32.561,506,11.62,0.0,349,555,13,2253,548,29458,774,2.776,2.406,27.708,3.117,553,65,488,599,638.6,2.35,0.0,86.9,4.7,124.2,872,1527,497,26.17,14.09,2.414,5659,5154,140.9,306,0,0.0,3008.08,0.026,0.025,0.150,0.188,0.911,0.781,13.663,6.788,0.915,27.105,0.420,0.022,28.405,3.365,0.000,0.360,0.007,0.026,0.062,1,870.0,870.0,0.0,1,1,1,1,2,0,2,0,2,0,2,0,1,1,2,0,2,0,2,0,2,0,2,1,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,1,1,2
3,83735.0,58191.539622,0.000000,1.0,22.0,2.0,1.0,15.0,6,1,1,12,1,2,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,12,1340,70.38,157.99,54.77,14.0,49.23,17.593,18.327,9.588,138,4.92,0.0,390,416,25,301,8,2272,227,0.866,1.608,24.306,1.259,175,63,110,218,227.8,3.08,0.0,9.8,6.9,36.3,1284,1439,318,9.07,7.59,0.822,2337,2180,104.5,248,104,0.0,1397.88,0.345,0.248,0.171,0.396,0.450,1.518,9.005,4.795,0.563,17.014,0.258,0.036,8.370,1.001,0.004,0.076,0.015,0.017,0.019,2,180.0,0.0,180.0,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,1,1,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2
4,83736.0,12358.085471,9813.677587,1.0,73.0,2.0,2.0,2.0,7,1,1,12,1,1,4,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,6,604,19.30,90.30,71.84,2.0,19.63,4.581,8.504,

In [102]:
tot_intake1_df.tail()

,SEQN,WTDRD1,WTDR2D,DR1DRSTZ,DR1EXMER,DRABF,DRDINT,DR1DBIH,DR1DAY,DR1LANG,DR1MRESP,DR1HELP,DBQ095Z,DBD100,DRQSPREP,DR1STY,DR1SKY,DRQSDIET,DRQSDT1,DRQSDT2,DRQSDT3,DRQSDT4,DRQSDT5,DRQSDT6,DRQSDT7,DRQSDT8,DRQSDT9,DRQSDT10,DRQSDT11,DRQSDT12,DRQSDT91,DR1TNUMF,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TATOA,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TB12A,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TTHEO,DR1TALCO,DR1TMOIS,DR1TS040,DR1TS060,DR1TS080,DR1TS100,DR1TS120,DR1TS140,DR1TS160,DR1TS180,DR1TM161,DR1TM181,DR1TM201,DR1TM221,DR1TP182,DR1TP183,DR1TP184,DR1TP204,DR1TP205,DR1TP225,DR1TP226,DR1_300,DR1_320Z,DR1_330Z,DR1BWATZ,DR1TWS,DRD340,DRD350A,DRD350AQ,DRD350B,DRD350BQ,DRD350C,DRD350CQ,DRD350D,DRD350DQ,DRD350E,DRD350EQ,DRD350F,DRD350FQ,DRD350G,DRD350GQ,DRD350H,DRD350HQ,DRD350I,DRD350IQ,DRD350J,DRD350JQ,DRD350K,DRD360,DRD370A,DRD370AQ,DRD370B,DRD370BQ,DRD370C,DRD370CQ,DRD370D,DRD370DQ,DRD370E,DRD370EQ,DRD370F,DRD370FQ,DRD370G,DRD370GQ,DRD370H,DRD370HQ,DRD370I,DRD370IQ,DRD370J,DRD370JQ,DRD370K,DRD370KQ,DRD370L,DRD370LQ,DRD370M,DRD370MQ,DRD370N,DRD370NQ,DRD370O,DRD370OQ,DRD370P,DRD370PQ,DRD370Q,DRD370QQ,DRD370R,DRD370RQ,DRD370S,DRD370SQ,DRD370T,DRD370TQ,DRD370U,DRD370UQ,DRD370V
9595,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,7,1,1,12,4,0,2,2,0,1,1,0,0,0,0,0,0,0,9,0,0,0,0,13,1083,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117,3.58,0.0,131,292,0,1943,1,6569,1828,0.57,1.088,11.335,1.269,163,0,163,163,192.7,2.18,0.0,35.1,1.7,125.1,525,830,262,6.88,5.55,1.175,1397,1778,66.7,192,0,31.8,1982.94,0.335,0.179,0.189,0.607,0.345,0.929,5.03,1.827,0.281,7.137,0.106,0.009,17.608,3.933,0.002,0.071,0.007,0.02,0.181,2,987.0,480.0,507.0,1,1,2,0,1,1,2,0,1,1,1,1,2,0,1,1,1,4,2,0,2,0,2,1,2,0,1,7,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,1,4,2,0,1,2,2,0,2,0,2,0,2,0,2,0,2,0,2
9596,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,7,1,1,12,4,0,2,2,0,1,1,0,0,0,0,0,0,0,9,0,0,0,0,13,1083,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117,3.58,0.0,131,292,0,1943,1,6569,1828,0.57,1.088,11.335,1.269,163,0,163,163,192.7,2.18,0.0,35.1,1.7,125.1,525,830,262,6.88,5.55,1.175,1397,1778,66.7,192,0,31.8,1982.94,0.335,0.179,0.189,0.607,0.345,0.929,5.03,1.827,0.281,7.137,0.106,0.009,17.608,3.933,0.002,0.071,0.007,0.02,0.181,2,987.0,480.0,507.0,1,1,2,0,1,1,2,0,1,1,1,1,2,0,1,1,1,4,2,0,2,0,2,1,2,0,1,7,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,1,4,2,0,1,2,2,0,2,0,2,0,2,0,2,0,2,0,2
9597,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,7,1,1,12,4,0,2,2,0,1,1,0,0,0,0,0,0,0,9,0,0,0,0,13,1083,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117,3.58,0.0,131,292,0,1943,1,6569,1828,0.57,1.088,11.335,1.269,163,0,163,163,192.7,2.18,0.0,35.1,1.7,125.1,525,830,262,6.88,5.55,1.175,1397,1778,66.7,192,0,31.8,1982.94,0.335,0.179,0.189,0.607,0.345,0.929,5.03,1.827,0.281,7.137,0.106,0.009,17.608,3.933,0.002,0.071,0.007,0.02,0.181,2,987.0,480.0,507.0,1,1,2,0,1,1,2,0,1,1,1,1,2,0,1,1,1,4,2,0,2,0,2,1,2,0,1,7,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,1,4,2,0,1,2,2,0,2,0,2,0,2,0,2,0,2,0,2
9598,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,7,1,1,12,4,0,2,2,0,1,1,0,0,0,0,0,0,0,9,0,0,0,0,13,1083,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117,3.58,0.0,131,292,0,1943,1,6569,1828,0.57,1.088,11.335,1.269,163,0,163,163,192.7,2.18,0.0,35.1,1.7,125.1,525,830,262,6.88,5.55,1.175,1397,1778,66.7,192,0,31.8,1982.94,0.335,0.179,0.189,0.607,0.345,0.929,5.03,1.827,0.281,7.137,0.106,0.009,17.608,3.933,0.002,0.071,0.007,0.02,0.181,2,987.0,480.0,507.0,1,1,2,0,1,1,2,0,1,1,1,1,2,0,1,1,1,4,2,0,2,0,2,1,2,0,1,7,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,1,4,2,0,1,2,2,0,2,0,2,0,2,0,2,0,2,0,

In [103]:
tot_intake1_df['SEQN'].nunique()

9545

In [104]:
tot_intake1_df = tot_intake1_df.drop(['DR1DRSTZ','DR1EXMER','DRABF','DR1DBIH',
                                      'DR1DAY', 'DR1LANG', 'DR1MRESP', 'DR1HELP',
                                      'DR1TNUMF', 'DR1TS040', 'DR1TS060', 'DR1TS080',
                                      'DR1TS100', 'DR1TS120', 'DR1TS140', 'DR1TS160', 
                                      'DR1TS180', 'DR1TM161', 'DR1TM181', 'DR1TM201', 
                                      'DR1TM221', 'DR1TP182', 'DR1TP183', 'DR1TP184', 
                                      'DR1TP204', 'DR1TP205', 'DR1TP225', 'DR1TP226', 
                                      'DR1_300', 'DR1_330Z', 'DR1BWATZ', 'DR1TWS', 
                                      'DRD340', 'DRD350A','DRD350AQ', 'DRD350B', 
                                      'DRD350BQ', 'DRD350C', 'DRD350CQ', 'DRD350D', 
                                      'DRD350DQ', 'DRD350E', 'DRD350EQ', 'DRD350F', 
                                      'DRD350FQ', 'DRD350G', 'DRD350GQ', 'DRD350H', 
                                      'DRD350HQ', 'DRD350I', 'DRD350IQ', 'DRD350J', 
                                      'DRD350JQ', 'DRD350K','DRD370AQ', 'DRD370A', 
                                      'DRD370B', 'DRD370BQ', 'DRD370C', 'DRD370CQ', 
                                      'DRD370D', 'DRD370DQ', 'DRD370E', 'DRD370EQ', 
                                      'DRD370F', 'DRD370FQ', 'DRD370G', 'DRD370GQ', 
                                      'DRD370H', 'DRD370HQ', 'DRD370I', 'DRD370IQ', 
                                      'DRD370J', 'DRD370JQ', 'DRD370K', 'DRD370KQ', 
                                      'DRD370L', 'DRD370LQ', 'DRD370M', 'DRD370MQ', 
                                      'DRD370N', 'DRD370NQ', 'DRD370O', 'DRD370OQ', 
                                      'DRD370P', 'DRD370PQ', 'DRD370Q', 'DRD370QQ', 
                                      'DRD370R', 'DRD370RQ', 'DRD370S', 'DRD370SQ', 
                                      'DRD370T', 'DRD370TQ', 'DRD370U', 'DRD370UQ', 
                                      'DRD370V'], axis=1)
tot_intake1_df.head()

,SEQN,WTDRD1,WTDR2D,DRDINT,DBQ095Z,DBD100,DRQSPREP,DR1STY,DR1SKY,DRQSDIET,DRQSDT1,DRQSDT2,DRQSDT3,DRQSDT4,DRQSDT5,DRQSDT6,DRQSDT7,DRQSDT8,DRQSDT9,DRQSDT10,DRQSDT11,DRQSDT12,DRQSDT91,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TATOA,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TB12A,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TTHEO,DR1TALCO,DR1TMOIS,DR1_320Z,DRD360
0,83732.0,92670.699919,69945.934107,2.0,1,3,3,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1781,76.03,193.29,42.31,23.6,79.24,23.430,31.897,18.528,138,10.59,0.0,230,307,336,624,282,1206,1120,2.344,1.949,20.401,2.279,450,163,286,564,290.7,1.89,0.0,110.5,2.6,139.0,623,1052,255,16.01,8.73,1.095,5298,2641,113.6,360,0,0.0,3028.29,960.0,1
1,83733.0,16454.105994,13190.268870,2.0,4,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2964,62.36,356.85,180.84,7.3,77.91,25.722,19.098,19.216,407,4.19,0.0,352,453,405,966,20,0,1279,1.859,2.478,29.444,1.823,478,112,365,556,672.7,2.92,0.0,30.2,4.6,87.6,594,1414,262,11.01,4.82,0.571,3164,2249,89.9,192,0,89.3,4024.25,507.0,1
2,83734.0,6529.619788,4928.422423,2.0,1,3,4,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,2482,116.50,304.53,62.87,35.1,91.97,23.122,28.666,32.561,506,11.62,0.0,349,555,13,2253,548,29458,774,2.776,2.406,27.708,3.117,553,65,488,599,638.6,2.35,0.0,86.9,4.7,124.2,872,1527,497,26.17,14.09,2.414,5659,5154,140.9,306,0,0.0,3008.08,870.0,1
3,83735.0,58191.539622,0.000000,1.0,1,2,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1340,70.38,157.99,54.77,14.0,49.23,17.593,18.327,9.588,138,4.92,0.0,390,416,25,301,8,2272,227,0.866,1.608,24.306,1.259,175,63,110,218,227.8,3.08,0.0,9.8,6.9,36.3,1284,1439,318,9.07,7.59,0.822,2337,2180,104.5,248,104,0.0,1397.88,180.0,1
4,83736.0,12358.085471,9813.677587,2.0,1,1,4,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,604,19.30,90.30,71.84,2.0,19.63,4.581,8.504,3.889,96,1.76,0.0,17,85,7,789,67,4497,47,0.168,0.219,7.340,0.409,52,6,46,56,69.2,0.36,0.0,38.0,0.0,9.1,72,255,42,1.83,1.54,0.224,982,585,27.7,0,0,0.0,818.85,0.0,2


#intake values can be 0!! can not change to Nan
#tot_intake1_df = tot_intake1_df.replace(0, np.nan)
#missing values ~1217

In [105]:
tot_intake1_df.isnull().sum()

SEQN        0
WTDRD1      0
WTDR2D      0
DRDINT      0
DBQ095Z     0
DBD100      0
DRQSPREP    0
DR1STY      0
DR1SKY      0
DRQSDIET    0
DRQSDT1     0
DRQSDT2     0
DRQSDT3     0
DRQSDT4     0
DRQSDT5     0
DRQSDT6     0
DRQSDT7     0
DRQSDT8     0
DRQSDT9     0
DRQSDT10    0
DRQSDT11    0
DRQSDT12    0
DRQSDT91    0
DR1TKCAL    0
DR1TPROT    0
DR1TCARB    0
DR1TSUGR    0
DR1TFIBE    0
DR1TTFAT    0
DR1TSFAT    0
DR1TMFAT    0
DR1TPFAT    0
DR1TCHOL    0
DR1TATOC    0
DR1TATOA    0
DR1TRET     0
DR1TVARA    0
DR1TACAR    0
DR1TBCAR    0
DR1TCRYP    0
DR1TLYCO    0
DR1TLZ      0
DR1TVB1     0
DR1TVB2     0
DR1TNIAC    0
DR1TVB6     0
DR1TFOLA    0
DR1TFA      0
DR1TFF      0
DR1TFDFE    0
DR1TCHL     0
DR1TVB12    0
DR1TB12A    0
DR1TVC      0
DR1TVD      0
DR1TVK      0
DR1TCALC    0
DR1TPHOS    0
DR1TMAGN    0
DR1TIRON    0
DR1TZINC    0
DR1TCOPP    0
DR1TSODI    0
DR1TPOTA    0
DR1TSELE    0
DR1TCAFF    0
DR1TTHEO    0
DR1TALCO    0
DR1TMOIS    0
DR1_320Z    0
DRD360      0
dtype:

In [106]:
tot_intake1_df = tot_intake1_df.drop(['DR1SKY', 'DRQSDT1', 'DRQSDT2', 
                                      'DRQSDT3', 'DRQSDT4', 'DRQSDT5',
                                      'DRQSDT6',  'DRQSDT7', 'DRQSDT8',
                                      'DRQSDT9',  'DRQSDT10', 'DRQSDT11', 
                                      'DRQSDT12', 'DRQSDT91', 'DR1TALCO',
                                     'DR1TATOA', 'DR1TB12A', 'DR1TTHEO'],
                                     axis=1)

In [107]:
tot_intake1_df.head()

,SEQN,WTDRD1,WTDR2D,DRDINT,DBQ095Z,DBD100,DRQSPREP,DR1STY,DRQSDIET,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TMOIS,DR1_320Z,DRD360
0,83732.0,92670.699919,69945.934107,2.0,1,3,3,1,2,1781,76.03,193.29,42.31,23.6,79.24,23.430,31.897,18.528,138,10.59,230,307,336,624,282,1206,1120,2.344,1.949,20.401,2.279,450,163,286,564,290.7,1.89,110.5,2.6,139.0,623,1052,255,16.01,8.73,1.095,5298,2641,113.6,360,3028.29,960.0,1
1,83733.0,16454.105994,13190.268870,2.0,4,0,2,2,2,2964,62.36,356.85,180.84,7.3,77.91,25.722,19.098,19.216,407,4.19,352,453,405,966,20,0,1279,1.859,2.478,29.444,1.823,478,112,365,556,672.7,2.92,30.2,4.6,87.6,594,1414,262,11.01,4.82,0.571,3164,2249,89.9,192,4024.25,507.0,1
2,83734.0,6529.619788,4928.422423,2.0,1,3,4,1,1,2482,116.50,304.53,62.87,35.1,91.97,23.122,28.666,32.561,506,11.62,349,555,13,2253,548,29458,774,2.776,2.406,27.708,3.117,553,65,488,599,638.6,2.35,86.9,4.7,124.2,872,1527,497,26.17,14.09,2.414,5659,5154,140.9,306,3008.08,870.0,1
3,83735.0,58191.539622,0.000000,1.0,1,2,2,2,2,1340,70.38,157.99,54.77,14.0,49.23,17.593,18.327,9.588,138,4.92,390,416,25,301,8,2272,227,0.866,1.608,24.306,1.259,175,63,110,218,227.8,3.08,9.8,6.9,36.3,1284,1439,318,9.07,7.59,0.822,2337,2180,104.5,248,1397.88,180.0,1
4,83736.0,12358.085471,9813.677587,2.0,1,1,4,2,2,604,19.30,90.30,71.84,2.0,19.63,4.581,8.504,3.889,96,1.76,17,85,7,789,67,4497,47,0.168,0.219,7.340,0.409,52,6,46,56,69.2,0.36,38.0,0.0,9.1,72,255,42,1.83,1.54,0.224,982,585,27.7,0,818.85,0.0,2


In [108]:
tot_intake1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9600 entries, 0 to 9599
Data columns (total 53 columns):
SEQN        9600 non-null float64
WTDRD1      9600 non-null float64
WTDR2D      9600 non-null float64
DRDINT      9600 non-null float64
DBQ095Z     9600 non-null int64
DBD100      9600 non-null int64
DRQSPREP    9600 non-null int64
DR1STY      9600 non-null int64
DRQSDIET    9600 non-null int64
DR1TKCAL    9600 non-null int64
DR1TPROT    9600 non-null float64
DR1TCARB    9600 non-null float64
DR1TSUGR    9600 non-null float64
DR1TFIBE    9600 non-null float64
DR1TTFAT    9600 non-null float64
DR1TSFAT    9600 non-null float64
DR1TMFAT    9600 non-null float64
DR1TPFAT    9600 non-null float64
DR1TCHOL    9600 non-null int64
DR1TATOC    9600 non-null float64
DR1TRET     9600 non-null int64
DR1TVARA    9600 non-null int64
DR1TACAR    9600 non-null int64
DR1TBCAR    9600 non-null int64
DR1TCRYP    9600 non-null int64
DR1TLYCO    9600 non-null int64
DR1TLZ      9600 non-null int64
DR1

## total dietary intake 2

In [109]:
tot_intake2_df.head()

,SEQN,WTDRD1,WTDR2D,DR2DRSTZ,DR2EXMER,DRABF,DRDINT,DR2DBIH,DR2DAY,DR2LANG,DR2MRESP,DR2HELP,DR2TNUMF,DR2STY,DR2SKY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TATOA,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TB12A,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TCAFF,DR2TTHEO,DR2TALCO,DR2TMOIS,DR2TS040,DR2TS060,DR2TS080,DR2TS100,DR2TS120,DR2TS140,DR2TS160,DR2TS180,DR2TM161,DR2TM181,DR2TM201,DR2TM221,DR2TP182,DR2TP183,DR2TP184,DR2TP204,DR2TP205,DR2TP225,DR2TP226,DR2_300,DR2_320Z,DR2_330Z,DR2BWATZ,DR2TWS
0,83732,92670.699919,69945.934107,1,87,2,2,2,4,1,1,12,25,1,1,2744,133.60,281.64,118.40,15.9,121.59,40.420,42.428,26.813,635,8.62,0.0,494,996,91,5970,42,1715,8474,2.027,3.286,35.661,3.283,395,160,235,507,540.4,7.01,0.0,64.6,9.6,533.4,1433,1880,334,15.67,15.09,1.265,4431,3774,167.8,171,7,0.0,3694.92,0.554,0.430,0.457,0.545,0.663,3.078,22.625,10.664,2.019,39.043,0.486,0.112,23.147,2.927,0.000,0.350,0.013,0.025,0.061,2,1440.0,480.0,960.0,2
1,83733,16454.105994,13190.268870,1,87,2,2,20,3,1,1,12,13,2,0,3971,165.86,223.57,34.52,12.2,154.55,32.969,61.039,47.020,773,17.82,0.0,231,296,144,644,108,3165,1133,1.933,2.806,43.695,3.068,529,126,403,617,1105.8,18.44,0.0,27.9,3.2,139.4,629,2662,524,15.95,9.30,1.213,4653,4915,153.5,180,0,144.9,5223.07,0.458,0.285,0.170,0.369,0.387,1.524,21.241,7.602,1.648,58.090,0.924,0.027,39.908,4.940,0.019,0.498,0.136,0.117,0.489,2,480.0,0.0,480.0,4
2,83734,6529.619788,4928.422423,1,43,2,2,9,3,1,5,12,15,2,0,1928,93.78,194.17,94.51,17.4,90.14,39.829,30.152,10.465,326,4.41,0.0,569,625,27,615,51,3383,226,0.960,4.828,21.310,1.742,199,83,117,256,391.0,5.48,0.0,23.4,0.3,63.9,887,1348,277,14.61,16.64,1.601,2625,2726,97.4,228,216,0.0,2096.38,1.193,0.923,0.498,1.073,1.424,4.640,19.094,9.613,1.592,27.459,0.215,0.017,9.087,1.016,0.063,0.151,0.009,0.043,0.001,2,720.0,720.0,0.0,1
3,83735,58191.539622,0.000000,5,0,2,1,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.0,0.00,0.000,0.000,0.000,0,0.00,0.0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0,0,0,0.00,0.00,0.000,0,0,0.0,0,0,0.0,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0.0,0.0,0.0,0
4,83736,12358.085471,9813.677587,1,87,2,2,6,4,1,1,12,8,2,0,1102,31.29,131.83,73.63,3.1,50.88,21.936,17.853,5.942,181,1.87,0.0,210,227,11,144,111,1051,144,0.587,0.297,11.569,0.591,134,92,42,199,95.2,0.67,0.0,52.7,0.1,6.8,116,431,58,3.41,3.05,0.286,1924,694,51.9,22,0,0.0,822.95,0.915,0.569,0.339,0.720,0.738,2.324,11.284,4.700,1.271,16.300,0.194,0.013,5.194,0.534,0.001,0.078,0.002,0.008,0.003,2,0.0,0.0,0.0,4


In [110]:
tot_intake2_df.tail()

,SEQN,WTDRD1,WTDR2D,DR2DRSTZ,DR2EXMER,DRABF,DRDINT,DR2DBIH,DR2DAY,DR2LANG,DR2MRESP,DR2HELP,DR2TNUMF,DR2STY,DR2SKY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TATOA,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TB12A,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TCAFF,DR2TTHEO,DR2TALCO,DR2TMOIS,DR2TS040,DR2TS060,DR2TS080,DR2TS100,DR2TS120,DR2TS140,DR2TS160,DR2TS180,DR2TM161,DR2TM181,DR2TM201,DR2TM221,DR2TP182,DR2TP183,DR2TP184,DR2TP204,DR2TP205,DR2TP225,DR2TP226,DR2_300,DR2_320Z,DR2_330Z,DR2BWATZ,DR2TWS
9595,93702,67560.380806,55166.938286,1,87,2,2,4,3,1,1,12,17,2,0,868,70.29,63.61,33.28,23.7,40.71,6.641,17.82,11.094,103,7.45,0.0,76,637,1228,6077,55,5356,5538,0.754,1.248,32.727,2.342,544,0,544,544,315.0,6.92,0.0,68.0,16.7,273.9,244,979,250,6.16,4.08,1.084,1282,3246,129.8,221,0,0.0,2431.34,0.046,0.029,0.04,0.057,0.045,0.405,4.865,0.965,1.013,15.856,0.721,0.069,8.66,1.346,0.1,0.035,0.267,0.069,0.544,2,900.0,900.0,0.0,1
9596,93702,67560.380806,55166.938286,1,87,2,2,4,3,1,1,12,17,2,0,868,70.29,63.61,33.28,23.7,40.71,6.641,17.82,11.094,103,7.45,0.0,76,637,1228,6077,55,5356,5538,0.754,1.248,32.727,2.342,544,0,544,544,315.0,6.92,0.0,68.0,16.7,273.9,244,979,250,6.16,4.08,1.084,1282,3246,129.8,221,0,0.0,2431.34,0.046,0.029,0.04,0.057,0.045,0.405,4.865,0.965,1.013,15.856,0.721,0.069,8.66,1.346,0.1,0.035,0.267,0.069,0.544,2,900.0,900.0,0.0,1
9597,93702,67560.380806,55166.938286,1,87,2,2,4,3,1,1,12,17,2,0,868,70.29,63.61,33.28,23.7,40.71,6.641,17.82,11.094,103,7.45,0.0,76,637,1228,6077,55,5356,5538,0.754,1.248,32.727,2.342,544,0,544,544,315.0,6.92,0.0,68.0,16.7,273.9,244,979,250,6.16,4.08,1.084,1282,3246,129.8,221,0,0.0,2431.34,0.046,0.029,0.04,0.057,0.045,0.405,4.865,0.965,1.013,15.856,0.721,0.069,8.66,1.346,0.1,0.035,0.267,0.069,0.544,2,900.0,900.0,0.0,1
9598,93702,67560.380806,55166.938286,1,87,2,2,4,3,1,1,12,17,2,0,868,70.29,63.61,33.28,23.7,40.71,6.641,17.82,11.094,103,7.45,0.0,76,637,1228,6077,55,5356,5538,0.754,1.248,32.727,2.342,544,0,544,544,315.0,6.92,0.0,68.0,16.7,273.9,244,979,250,6.16,4.08,1.084,1282,3246,129.8,221,0,0.0,2431.34,0.046,0.029,0.04,0.057,0.045,0.405,4.865,0.965,1.013,15.856,0.721,0.069,8.66,1.346,0.1,0.035,0.267,0.069,0.544,2,900.0,900.0,0.0,1
9599,93702,67560.380806,55166.938286,1,87,2,2,4,3,1,1,12,17,2,0,868,70.29,63.61,33.28,23.7,40.71,6.641,17.82,11.094,103,7.45,0.0,76,637,1228,6077,55,5356,5538,0.754,1.248,32.727,2.342,544,0,544,544,315.0,6.92,0.0,68.0,16.7,273.9,244,979,250,6.16,4.08,1.084,1282,3246,129.8,221,0,0.0,2431.34,0.046,0.029,0.04,0.057,0.045,0.405,4.865,0.965,1.013,15.856,0.721,0.069,8.66,1.346,0.1,0.035,0.267,0.069,0.544,2,900.0,900.0,0.0,1


In [111]:
tot_intake2_df['SEQN'].nunique()

9544

In [112]:
tot_intake2_df = tot_intake2_df.drop(['DR2DRSTZ','DR2EXMER','DRABF','DR2DBIH',
                                      'DR2DAY', 'DR2LANG', 'DR2MRESP', 'DR2HELP',
                                      'DR2TNUMF', 'DR2TS040', 'DR2TS060', 'DR2TS080',
                                      'DR2TS100', 'DR2TS120', 'DR2TS140', 'DR2TS160', 
                                      'DR2TS180', 'DR2TM161', 'DR2TM181', 'DR2TM201', 
                                      'DR2TM221', 'DR2TP182', 'DR2TP183', 'DR2TP184', 
                                      'DR2TP204', 'DR2TP205', 'DR2TP225', 'DR2TP226', 
                                      'DR2_300', 'DR2_330Z', 'DR2BWATZ', 'DR2TWS'], axis=1)
tot_intake2_df.head()

,SEQN,WTDRD1,WTDR2D,DRDINT,DR2STY,DR2SKY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TATOA,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TB12A,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TCAFF,DR2TTHEO,DR2TALCO,DR2TMOIS,DR2_320Z
0,83732,92670.699919,69945.934107,2,1,1,2744,133.60,281.64,118.40,15.9,121.59,40.420,42.428,26.813,635,8.62,0.0,494,996,91,5970,42,1715,8474,2.027,3.286,35.661,3.283,395,160,235,507,540.4,7.01,0.0,64.6,9.6,533.4,1433,1880,334,15.67,15.09,1.265,4431,3774,167.8,171,7,0.0,3694.92,1440.0
1,83733,16454.105994,13190.268870,2,2,0,3971,165.86,223.57,34.52,12.2,154.55,32.969,61.039,47.020,773,17.82,0.0,231,296,144,644,108,3165,1133,1.933,2.806,43.695,3.068,529,126,403,617,1105.8,18.44,0.0,27.9,3.2,139.4,629,2662,524,15.95,9.30,1.213,4653,4915,153.5,180,0,144.9,5223.07,480.0
2,83734,6529.619788,4928.422423,2,2,0,1928,93.78,194.17,94.51,17.4,90.14,39.829,30.152,10.465,326,4.41,0.0,569,625,27,615,51,3383,226,0.960,4.828,21.310,1.742,199,83,117,256,391.0,5.48,0.0,23.4,0.3,63.9,887,1348,277,14.61,16.64,1.601,2625,2726,97.4,228,216,0.0,2096.38,720.0
3,83735,58191.539622,0.000000,1,0,0,0,0.00,0.00,0.00,0.0,0.00,0.000,0.000,0.000,0,0.00,0.0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0,0,0,0.00,0.00,0.000,0,0,0.0,0,0,0.0,0.00,0.0
4,83736,12358.085471,9813.677587,2,2,0,1102,31.29,131.83,73.63,3.1,50.88,21.936,17.853,5.942,181,1.87,0.0,210,227,11,144,111,1051,144,0.587,0.297,11.569,0.591,134,92,42,199,95.2,0.67,0.0,52.7,0.1,6.8,116,431,58,3.41,3.05,0.286,1924,694,51.9,22,0,0.0,822.95,0.0


#intake values can be 0!! can not change to Nan
#tot_intake2_df = tot_intake2_df.replace(0, np.nan)

In [113]:
tot_intake2_df = tot_intake2_df.drop(['DR2SKY', 'DR2TATOA', 'DR2TB12A', 'DR2TCAFF',
                                      'DR2TTHEO', 'DR2TALCO'], axis=1)

In [114]:
tot_intake2_df.isnull().sum()

SEQN        0
WTDRD1      0
WTDR2D      0
DRDINT      0
DR2STY      0
DR2TKCAL    0
DR2TPROT    0
DR2TCARB    0
DR2TSUGR    0
DR2TFIBE    0
DR2TTFAT    0
DR2TSFAT    0
DR2TMFAT    0
DR2TPFAT    0
DR2TCHOL    0
DR2TATOC    0
DR2TRET     0
DR2TVARA    0
DR2TACAR    0
DR2TBCAR    0
DR2TCRYP    0
DR2TLYCO    0
DR2TLZ      0
DR2TVB1     0
DR2TVB2     0
DR2TNIAC    0
DR2TVB6     0
DR2TFOLA    0
DR2TFA      0
DR2TFF      0
DR2TFDFE    0
DR2TCHL     0
DR2TVB12    0
DR2TVC      0
DR2TVD      0
DR2TVK      0
DR2TCALC    0
DR2TPHOS    0
DR2TMAGN    0
DR2TIRON    0
DR2TZINC    0
DR2TCOPP    0
DR2TSODI    0
DR2TPOTA    0
DR2TSELE    0
DR2TMOIS    0
DR2_320Z    0
dtype: int64

In [115]:
tot_intake2_df['SEQN'] = tot_intake2_df['SEQN'].astype(float)

In [116]:
tot_intake2_df.head()

,SEQN,WTDRD1,WTDR2D,DRDINT,DR2STY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TMOIS,DR2_320Z
0,83732.0,92670.699919,69945.934107,2,1,2744,133.60,281.64,118.40,15.9,121.59,40.420,42.428,26.813,635,8.62,494,996,91,5970,42,1715,8474,2.027,3.286,35.661,3.283,395,160,235,507,540.4,7.01,64.6,9.6,533.4,1433,1880,334,15.67,15.09,1.265,4431,3774,167.8,3694.92,1440.0
1,83733.0,16454.105994,13190.268870,2,2,3971,165.86,223.57,34.52,12.2,154.55,32.969,61.039,47.020,773,17.82,231,296,144,644,108,3165,1133,1.933,2.806,43.695,3.068,529,126,403,617,1105.8,18.44,27.9,3.2,139.4,629,2662,524,15.95,9.30,1.213,4653,4915,153.5,5223.07,480.0
2,83734.0,6529.619788,4928.422423,2,2,1928,93.78,194.17,94.51,17.4,90.14,39.829,30.152,10.465,326,4.41,569,625,27,615,51,3383,226,0.960,4.828,21.310,1.742,199,83,117,256,391.0,5.48,23.4,0.3,63.9,887,1348,277,14.61,16.64,1.601,2625,2726,97.4,2096.38,720.0
3,83735.0,58191.539622,0.000000,1,0,0,0.00,0.00,0.00,0.0,0.00,0.000,0.000,0.000,0,0.00,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0,0,0,0.00,0.00,0.000,0,0,0.0,0.00,0.0
4,83736.0,12358.085471,9813.677587,2,2,1102,31.29,131.83,73.63,3.1,50.88,21.936,17.853,5.942,181,1.87,210,227,11,144,111,1051,144,0.587,0.297,11.569,0.591,134,92,42,199,95.2,0.67,52.7,0.1,6.8,116,431,58,3.41,3.05,0.286,1924,694,51.9,822.95,0.0


In [117]:
tot_intake2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9600 entries, 0 to 9599
Data columns (total 47 columns):
SEQN        9600 non-null float64
WTDRD1      9600 non-null float64
WTDR2D      9600 non-null float64
DRDINT      9600 non-null int64
DR2STY      9600 non-null int64
DR2TKCAL    9600 non-null int64
DR2TPROT    9600 non-null float64
DR2TCARB    9600 non-null float64
DR2TSUGR    9600 non-null float64
DR2TFIBE    9600 non-null float64
DR2TTFAT    9600 non-null float64
DR2TSFAT    9600 non-null float64
DR2TMFAT    9600 non-null float64
DR2TPFAT    9600 non-null float64
DR2TCHOL    9600 non-null int64
DR2TATOC    9600 non-null float64
DR2TRET     9600 non-null int64
DR2TVARA    9600 non-null int64
DR2TACAR    9600 non-null int64
DR2TBCAR    9600 non-null int64
DR2TCRYP    9600 non-null int64
DR2TLYCO    9600 non-null int64
DR2TLZ      9600 non-null int64
DR2TVB1     9600 non-null float64
DR2TVB2     9600 non-null float64
DR2TNIAC    9600 non-null float64
DR2TVB6     9600 non-null float

## total cholesterol

In [118]:
tot_chol_df.head()

,SEQN,LBXTC,LBDTCSI
0,83732.0,173.0,4.47
1,83733.0,265.0,6.85
2,83734.0,229.0,5.92
3,83735.0,174.0,4.50
4,83736.0,204.0,5.28


In [119]:
tot_chol_df.tail()

,SEQN,LBXTC,LBDTCSI
8095,3.687825e-40,3.687825e-40,3.687825e-40
8096,3.687825e-40,3.687825e-40,3.687825e-40
8097,3.687825e-40,3.687825e-40,3.687825e-40
8098,3.687825e-40,3.687825e-40,3.687825e-40
8099,3.687825e-40,3.687825e-40,3.687825e-40


In [120]:
tot_chol_df['SEQN'].nunique()

8022

In [121]:
tot_chol_df = tot_chol_df.drop(['LBDTCSI'], axis=1)
tot_chol_df.head()

,SEQN,LBXTC
0,83732.0,173.0
1,83733.0,265.0
2,83734.0,229.0
3,83735.0,174.0
4,83736.0,204.0


In [122]:
tot_chol_df = tot_chol_df.replace(0, np.nan)

In [123]:
tot_chol_df.isnull().sum()

SEQN       0
LBXTC    765
dtype: int64

In [124]:
tot_chol_df.head()

,SEQN,LBXTC
0,83732.0,173.0
1,83733.0,265.0
2,83734.0,229.0
3,83735.0,174.0
4,83736.0,204.0


In [125]:
tot_chol_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8100 entries, 0 to 8099
Data columns (total 2 columns):
SEQN     8100 non-null float64
LBXTC    7335 non-null float64
dtypes: float64(2)
memory usage: 126.6 KB


## total dietary supplement day 1

In [126]:
tot_diet_supp_1_df.head()

,SEQN,WTDRD1,WTDR2D,DR1DRSTZ,DR1EXMER,DRDINT,DR1DBIH,DR1DAY,DR1LANG,DR1MRESP,DR1HELP,DS1DSCNT,DS1DS,DS1ANCNT,DS1AN,DS1TKCAL,DS1TPROT,DS1TCARB,DS1TSUGR,DS1TFIBE,DS1TTFAT,DS1TSFAT,DS1TMFAT,DS1TPFAT,DS1TCHOL,DS1TLYCO,DS1TLZ,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TCHL,DS1TVC,DS1TVK,DS1TVD,DS1TCALC,DS1TPHOS,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE,DS1TCAFF,DS1TIODI
0,83732.0,92670.699919,69945.934107,1.0,71,2,-1,1,1,1,12,2,1,0,2,150,0.0,42.0,24.0,18.0,0.0,0.0,0.0,0.0,0,300,250,1.50,1.7,20.0,3.0,25.0,400,680,0.0,60.0,30.0,12.5,220.0,20.0,50.0,4.2,11.0,0.5,30.0,290.0,19.0,0,150.0
1,83733.0,16454.105994,13190.268870,1.0,61,2,17,7,1,1,12,0,2,1,1,5,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,400.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,83734.0,6529.619788,4928.422423,1.0,14,2,6,7,1,1,12,1,1,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,300,0,1.35,1.7,18.0,3.0,18.0,400,680,0.0,60.0,20.0,17.5,210.0,0.0,140.0,0.0,15.0,2.0,0.0,0.0,110.0,0,0.0
3,83735.0,58191.539622,0.000000,1.0,22,1,15,6,1,1,12,0,2,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,83736.0,12358.085471,9813.677587,1.0,73,2,2,7,1,1,12,1,1,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.0,0.0,26.0,25.0,1000,1700,0.0,90.0,0.0,10.0,155.0,0.0,50.0,18.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [127]:
tot_diet_supp_1_df.tail()

,SEQN,WTDRD1,WTDR2D,DR1DRSTZ,DR1EXMER,DRDINT,DR1DBIH,DR1DAY,DR1LANG,DR1MRESP,DR1HELP,DS1DSCNT,DS1DS,DS1ANCNT,DS1AN,DS1TKCAL,DS1TPROT,DS1TCARB,DS1TSUGR,DS1TFIBE,DS1TTFAT,DS1TSFAT,DS1TMFAT,DS1TPFAT,DS1TCHOL,DS1TLYCO,DS1TLZ,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TCHL,DS1TVC,DS1TVK,DS1TVD,DS1TCALC,DS1TPHOS,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE,DS1TCAFF,DS1TIODI
9595,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,49,2,1,7,1,1,12,0,2,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
9596,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,49,2,1,7,1,1,12,0,2,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
9597,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,49,2,1,7,1,1,12,0,2,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
9598,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,49,2,1,7,1,1,12,0,2,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
9599,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,49,2,1,7,1,1,12,0,2,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [128]:
tot_diet_supp_1_df['SEQN'].nunique()

9545

In [129]:
tot_diet_supp_1_df = tot_diet_supp_1_df.drop(['DR1DRSTZ','DR1EXMER','DRDINT','DR1DBIH',
                                      'DR1DAY', 'DR1LANG', 'DR1MRESP', 'DR1HELP',
                                      'DS1DSCNT', 'DS1ANCNT'], axis=1)
tot_diet_supp_1_df.head()

,SEQN,WTDRD1,WTDR2D,DS1DS,DS1AN,DS1TKCAL,DS1TPROT,DS1TCARB,DS1TSUGR,DS1TFIBE,DS1TTFAT,DS1TSFAT,DS1TMFAT,DS1TPFAT,DS1TCHOL,DS1TLYCO,DS1TLZ,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TCHL,DS1TVC,DS1TVK,DS1TVD,DS1TCALC,DS1TPHOS,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE,DS1TCAFF,DS1TIODI
0,83732.0,92670.699919,69945.934107,1,2,150,0.0,42.0,24.0,18.0,0.0,0.0,0.0,0.0,0,300,250,1.50,1.7,20.0,3.0,25.0,400,680,0.0,60.0,30.0,12.5,220.0,20.0,50.0,4.2,11.0,0.5,30.0,290.0,19.0,0,150.0
1,83733.0,16454.105994,13190.268870,2,1,5,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,400.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,83734.0,6529.619788,4928.422423,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,300,0,1.35,1.7,18.0,3.0,18.0,400,680,0.0,60.0,20.0,17.5,210.0,0.0,140.0,0.0,15.0,2.0,0.0,0.0,110.0,0,0.0
3,83735.0,58191.539622,0.000000,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,83736.0,12358.085471,9813.677587,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.0,0.0,26.0,25.0,1000,1700,0.0,90.0,0.0,10.0,155.0,0.0,50.0,18.0,0.0,0.0,0.0,0.0,0.0,0,0.0


#intake values can be 0!! can not change to Nan
#tot_diet_supp_1_df = tot_diet_supp_1_df.replace(0, np.nan)

In [130]:
tot_diet_supp_1_df.isnull().sum()

SEQN        0
WTDRD1      0
WTDR2D      0
DS1DS       0
DS1AN       0
DS1TKCAL    0
DS1TPROT    0
DS1TCARB    0
DS1TSUGR    0
DS1TFIBE    0
DS1TTFAT    0
DS1TSFAT    0
DS1TMFAT    0
DS1TPFAT    0
DS1TCHOL    0
DS1TLYCO    0
DS1TLZ      0
DS1TVB1     0
DS1TVB2     0
DS1TNIAC    0
DS1TVB6     0
DS1TVB12    0
DS1TFA      0
DS1TFDFE    0
DS1TCHL     0
DS1TVC      0
DS1TVK      0
DS1TVD      0
DS1TCALC    0
DS1TPHOS    0
DS1TMAGN    0
DS1TIRON    0
DS1TZINC    0
DS1TCOPP    0
DS1TSODI    0
DS1TPOTA    0
DS1TSELE    0
DS1TCAFF    0
DS1TIODI    0
dtype: int64

In [131]:
tot_diet_supp_1_df = tot_diet_supp_1_df.drop(['DS1TKCAL', 'DS1TPROT', 
                                              'DS1TCARB', 'DS1TSUGR', 
                                              'DS1TFIBE', 'DS1TTFAT',
                                              'DS1TSFAT', 'DS1TMFAT',
                                              'DS1TPFAT', 'DS1TCHOL',
                                              'DS1TLYCO',  'DS1TLZ',
                                              'DS1TCHL',  'DS1TVK',
                                              'DS1TPHOS', 'DS1TCAFF',
                                              'DS1TIODI'], axis=1)

In [132]:
tot_diet_supp_1_df.head()

,SEQN,WTDRD1,WTDR2D,DS1DS,DS1AN,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TVC,DS1TVD,DS1TCALC,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE
0,83732.0,92670.699919,69945.934107,1,2,1.50,1.7,20.0,3.0,25.0,400,680,60.0,12.5,220.0,50.0,4.2,11.0,0.5,30.0,290.0,19.0
1,83733.0,16454.105994,13190.268870,2,1,0.00,0.0,0.0,0.0,0.0,0,0,0.0,0.0,400.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,83734.0,6529.619788,4928.422423,1,2,1.35,1.7,18.0,3.0,18.0,400,680,60.0,17.5,210.0,140.0,0.0,15.0,2.0,0.0,0.0,110.0
3,83735.0,58191.539622,0.000000,2,2,0.00,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,83736.0,12358.085471,9813.677587,1,2,0.00,0.0,0.0,26.0,25.0,1000,1700,90.0,10.0,155.0,50.0,18.0,0.0,0.0,0.0,0.0,0.0


In [133]:
tot_diet_supp_1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9600 entries, 0 to 9599
Data columns (total 22 columns):
SEQN        9600 non-null float64
WTDRD1      9600 non-null float64
WTDR2D      9600 non-null float64
DS1DS       9600 non-null int64
DS1AN       9600 non-null int64
DS1TVB1     9600 non-null float64
DS1TVB2     9600 non-null float64
DS1TNIAC    9600 non-null float64
DS1TVB6     9600 non-null float64
DS1TVB12    9600 non-null float64
DS1TFA      9600 non-null int64
DS1TFDFE    9600 non-null int64
DS1TVC      9600 non-null float64
DS1TVD      9600 non-null float64
DS1TCALC    9600 non-null float64
DS1TMAGN    9600 non-null float64
DS1TIRON    9600 non-null float64
DS1TZINC    9600 non-null float64
DS1TCOPP    9600 non-null float64
DS1TSODI    9600 non-null float64
DS1TPOTA    9600 non-null float64
DS1TSELE    9600 non-null float64
dtypes: float64(18), int64(4)
memory usage: 1.6 MB


## total dietary supplement day 2

In [134]:
tot_diet_supp_2_df.head()

,SEQN,WTDRD1,WTDR2D,DR2DRSTZ,DR2EXMER,DRDINT,DR2DBIH,DR2DAY,DR2LANG,DR2MRESP,DR2HELP,DS2DSCNT,DS2DS,DS2ANCNT,DS2AN,DS2TKCAL,DS2TPROT,DS2TCARB,DS2TSUGR,DS2TFIBE,DS2TTFAT,DS2TSFAT,DS2TMFAT,DS2TPFAT,DS2TCHOL,DS2TLYCO,DS2TLZ,DS2TVB1,DS2TVB2,DS2TNIAC,DS2TVB6,DS2TVB12,DS2TFA,DS2TFDFE,DS2TCHL,DS2TVC,DS2TVK,DS2TVD,DS2TCALC,DS2TPHOS,DS2TMAGN,DS2TIRON,DS2TZINC,DS2TCOPP,DS2TSODI,DS2TPOTA,DS2TSELE,DS2TCAFF,DS2TIODI
0,83732.0,92670.699919,69945.934107,1.0,87,2,2,4,1,1,12,2,1,0,2,75,0.0,21.0,12.0,9.0,0.0,0.0,0.0,0.0,0.0,300,250,1.50,1.7,20.0,3.0,25.0,400,680,0.0,60.0,30.0,12.5,220.0,20.0,50.0,2.1,11.0,0.5,15.0,185.0,19.0,0,150.0
1,83733.0,16454.105994,13190.268870,1.0,87,2,20,3,1,1,12,0,2,1,1,5,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,400.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,83734.0,6529.619788,4928.422423,1.0,43,2,9,3,1,5,12,1,1,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300,0,1.35,1.7,18.0,3.0,18.0,400,680,0.0,60.0,20.0,17.5,210.0,0.0,140.0,0.0,15.0,2.0,0.0,0.0,110.0,0,0.0
3,83735.0,58191.539622,0.000000,5.0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,83736.0,12358.085471,9813.677587,1.0,87,2,6,4,1,1,12,1,1,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.0,0.0,26.0,25.0,1000,1700,0.0,90.0,0.0,10.0,155.0,0.0,50.0,18.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [135]:
tot_diet_supp_2_df.tail()

,SEQN,WTDRD1,WTDR2D,DR2DRSTZ,DR2EXMER,DRDINT,DR2DBIH,DR2DAY,DR2LANG,DR2MRESP,DR2HELP,DS2DSCNT,DS2DS,DS2ANCNT,DS2AN,DS2TKCAL,DS2TPROT,DS2TCARB,DS2TSUGR,DS2TFIBE,DS2TTFAT,DS2TSFAT,DS2TMFAT,DS2TPFAT,DS2TCHOL,DS2TLYCO,DS2TLZ,DS2TVB1,DS2TVB2,DS2TNIAC,DS2TVB6,DS2TVB12,DS2TFA,DS2TFDFE,DS2TCHL,DS2TVC,DS2TVK,DS2TVD,DS2TCALC,DS2TPHOS,DS2TMAGN,DS2TIRON,DS2TZINC,DS2TCOPP,DS2TSODI,DS2TPOTA,DS2TSELE,DS2TCAFF,DS2TIODI
9595,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,87,2,4,3,1,1,12,0,2,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
9596,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,87,2,4,3,1,1,12,0,2,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
9597,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,87,2,4,3,1,1,12,0,2,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
9598,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,87,2,4,3,1,1,12,0,2,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
9599,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,87,2,4,3,1,1,12,0,2,0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [136]:
tot_diet_supp_2_df['SEQN'].nunique()

9545

In [137]:
tot_diet_supp_2_df = tot_diet_supp_2_df.drop(['DR2DRSTZ','DR2EXMER','DRDINT','DR2DBIH',
                                      'DR2DAY', 'DR2LANG', 'DR2MRESP', 'DR2HELP',
                                      'DS2DSCNT', 'DS2ANCNT'], axis=1)
tot_diet_supp_2_df.head()

,SEQN,WTDRD1,WTDR2D,DS2DS,DS2AN,DS2TKCAL,DS2TPROT,DS2TCARB,DS2TSUGR,DS2TFIBE,DS2TTFAT,DS2TSFAT,DS2TMFAT,DS2TPFAT,DS2TCHOL,DS2TLYCO,DS2TLZ,DS2TVB1,DS2TVB2,DS2TNIAC,DS2TVB6,DS2TVB12,DS2TFA,DS2TFDFE,DS2TCHL,DS2TVC,DS2TVK,DS2TVD,DS2TCALC,DS2TPHOS,DS2TMAGN,DS2TIRON,DS2TZINC,DS2TCOPP,DS2TSODI,DS2TPOTA,DS2TSELE,DS2TCAFF,DS2TIODI
0,83732.0,92670.699919,69945.934107,1,2,75,0.0,21.0,12.0,9.0,0.0,0.0,0.0,0.0,0.0,300,250,1.50,1.7,20.0,3.0,25.0,400,680,0.0,60.0,30.0,12.5,220.0,20.0,50.0,2.1,11.0,0.5,15.0,185.0,19.0,0,150.0
1,83733.0,16454.105994,13190.268870,2,1,5,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,400.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,83734.0,6529.619788,4928.422423,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300,0,1.35,1.7,18.0,3.0,18.0,400,680,0.0,60.0,20.0,17.5,210.0,0.0,140.0,0.0,15.0,2.0,0.0,0.0,110.0,0,0.0
3,83735.0,58191.539622,0.000000,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,83736.0,12358.085471,9813.677587,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.00,0.0,0.0,26.0,25.0,1000,1700,0.0,90.0,0.0,10.0,155.0,0.0,50.0,18.0,0.0,0.0,0.0,0.0,0.0,0,0.0


#intake values can be 0!! can not change to Nan
#tot_diet_supp_2_df = tot_diet_supp_2_df.replace(0, np.nan)

In [138]:
tot_diet_supp_2_df.isnull().sum()

SEQN        0
WTDRD1      0
WTDR2D      0
DS2DS       0
DS2AN       0
DS2TKCAL    0
DS2TPROT    0
DS2TCARB    0
DS2TSUGR    0
DS2TFIBE    0
DS2TTFAT    0
DS2TSFAT    0
DS2TMFAT    0
DS2TPFAT    0
DS2TCHOL    0
DS2TLYCO    0
DS2TLZ      0
DS2TVB1     0
DS2TVB2     0
DS2TNIAC    0
DS2TVB6     0
DS2TVB12    0
DS2TFA      0
DS2TFDFE    0
DS2TCHL     0
DS2TVC      0
DS2TVK      0
DS2TVD      0
DS2TCALC    0
DS2TPHOS    0
DS2TMAGN    0
DS2TIRON    0
DS2TZINC    0
DS2TCOPP    0
DS2TSODI    0
DS2TPOTA    0
DS2TSELE    0
DS2TCAFF    0
DS2TIODI    0
dtype: int64

In [139]:
tot_diet_supp_2_df = tot_diet_supp_2_df.drop(['DS2TKCAL', 'DS2TPROT', 
                                              'DS2TCARB', 'DS2TSUGR', 
                                              'DS2TFIBE', 'DS2TTFAT',
                                              'DS2TSFAT', 'DS2TMFAT',
                                              'DS2TPFAT', 'DS2TCHOL',
                                              'DS2TLYCO',  'DS2TLZ',
                                              'DS2TCHL',  'DS2TVK',
                                              'DS2TPHOS', 'DS2TCAFF',
                                              'DS2TIODI'], axis=1)

In [140]:
tot_diet_supp_2_df.head()

,SEQN,WTDRD1,WTDR2D,DS2DS,DS2AN,DS2TVB1,DS2TVB2,DS2TNIAC,DS2TVB6,DS2TVB12,DS2TFA,DS2TFDFE,DS2TVC,DS2TVD,DS2TCALC,DS2TMAGN,DS2TIRON,DS2TZINC,DS2TCOPP,DS2TSODI,DS2TPOTA,DS2TSELE
0,83732.0,92670.699919,69945.934107,1,2,1.50,1.7,20.0,3.0,25.0,400,680,60.0,12.5,220.0,50.0,2.1,11.0,0.5,15.0,185.0,19.0
1,83733.0,16454.105994,13190.268870,2,1,0.00,0.0,0.0,0.0,0.0,0,0,0.0,0.0,400.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,83734.0,6529.619788,4928.422423,1,2,1.35,1.7,18.0,3.0,18.0,400,680,60.0,17.5,210.0,140.0,0.0,15.0,2.0,0.0,0.0,110.0
3,83735.0,58191.539622,0.000000,0,0,0.00,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,83736.0,12358.085471,9813.677587,1,2,0.00,0.0,0.0,26.0,25.0,1000,1700,90.0,10.0,155.0,50.0,18.0,0.0,0.0,0.0,0.0,0.0


In [141]:
tot_diet_supp_2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9600 entries, 0 to 9599
Data columns (total 22 columns):
SEQN        9600 non-null float64
WTDRD1      9600 non-null float64
WTDR2D      9600 non-null float64
DS2DS       9600 non-null int64
DS2AN       9600 non-null int64
DS2TVB1     9600 non-null float64
DS2TVB2     9600 non-null float64
DS2TNIAC    9600 non-null float64
DS2TVB6     9600 non-null float64
DS2TVB12    9600 non-null float64
DS2TFA      9600 non-null int64
DS2TFDFE    9600 non-null int64
DS2TVC      9600 non-null float64
DS2TVD      9600 non-null float64
DS2TCALC    9600 non-null float64
DS2TMAGN    9600 non-null float64
DS2TIRON    9600 non-null float64
DS2TZINC    9600 non-null float64
DS2TCOPP    9600 non-null float64
DS2TSODI    9600 non-null float64
DS2TPOTA    9600 non-null float64
DS2TSELE    9600 non-null float64
dtypes: float64(18), int64(4)
memory usage: 1.6 MB


## wt_hx_df

In [142]:
wt_hx_df.head()

,SEQN,WHD010,WHD020,WHQ030,WHQ040,WHD050,WHQ060,WHQ070,WHD080A,WHD080B,WHD080C,WHD080D,WHD080E,WHD080F,WHD080G,WHD080H,WHD080I,WHD080J,WHD080K,WHD080M,WHD080N,WHD080O,WHD080P,WHD080Q,WHD080R,WHD080S,WHD080T,WHD080U,WHD080L,WHQ225,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ190,WHQ200
0,83732.0,74,212,3,2,225,1,0,10,0,0,13,0,0,0,0,0,0,0,34,0,41,0,43,0,0,46,0,0,2,230,205,75,260,20,2,0
1,83733.0,68,193,1,2,191,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,180,160,68,205,51,2,0
2,83734.0,69,182,3,2,212,1,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43,0,0,0,0,0,1,224,165,70,240,46,2,0
3,83735.0,64,220,1,2,220,0,1,10,11,12,0,14,15,16,0,0,32,0,0,0,0,0,43,0,0,46,0,0,2,160,120,64,240,55,2,0
4,83736.0,64,125,2,1,135,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,127,127,0,135,33,2,0


In [143]:
wt_hx_df.tail()

,SEQN,WHD010,WHD020,WHQ030,WHQ040,WHD050,WHQ060,WHQ070,WHD080A,WHD080B,WHD080C,WHD080D,WHD080E,WHD080F,WHD080G,WHD080H,WHD080I,WHD080J,WHD080K,WHD080M,WHD080N,WHD080O,WHD080P,WHD080Q,WHD080R,WHD080S,WHD080T,WHD080U,WHD080L,WHQ225,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ190,WHQ200
6395,3.687825e-40,65,128,3,3,128,0,1,10,11,12,13,0,0,0,0,0,0,0,34,0,41,0,43,44,45,46,0,0,5,0,0,0,130,22,2,0
6396,3.687825e-40,65,128,3,3,128,0,1,10,11,12,13,0,0,0,0,0,0,0,34,0,41,0,43,44,45,46,0,0,5,0,0,0,130,22,2,0
6397,3.687825e-40,65,128,3,3,128,0,1,10,11,12,13,0,0,0,0,0,0,0,34,0,41,0,43,44,45,46,0,0,5,0,0,0,130,22,2,0
6398,3.687825e-40,65,128,3,3,128,0,1,10,11,12,13,0,0,0,0,0,0,0,34,0,41,0,43,44,45,46,0,0,5,0,0,0,130,22,2,0
6399,3.687825e-40,65,128,3,3,128,0,1,10,11,12,13,0,0,0,0,0,0,0,34,0,41,0,43,44,45,46,0,0,5,0,0,0,130,22,2,0


In [144]:
wt_hx_df['SEQN'].nunique()

6328

In [145]:
wt_hx_df = wt_hx_df.drop(['WHD010','WHQ030','WHQ040','WHQ070',
                          'WHD080D', 'WHD080P', 'WHD080L', 'WHD130'], axis=1)
wt_hx_df.head()

,SEQN,WHD020,WHD050,WHQ060,WHD080A,WHD080B,WHD080C,WHD080E,WHD080F,WHD080G,WHD080H,WHD080I,WHD080J,WHD080K,WHD080M,WHD080N,WHD080O,WHD080Q,WHD080R,WHD080S,WHD080T,WHD080U,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,WHQ200
0,83732.0,212,225,1,10,0,0,0,0,0,0,0,0,0,34,0,41,43,0,0,46,0,2,230,205,260,20,2,0
1,83733.0,193,191,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,180,160,205,51,2,0
2,83734.0,182,212,1,10,0,0,0,0,0,0,0,0,0,0,0,0,43,0,0,0,0,1,224,165,240,46,2,0
3,83735.0,220,220,0,10,11,12,14,15,16,0,0,32,0,0,0,0,43,0,0,46,0,2,160,120,240,55,2,0
4,83736.0,125,135,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,127,127,135,33,2,0


In [146]:
wt_hx_df = wt_hx_df.replace(0, np.nan)

In [147]:
wt_hx_df.isnull().sum()

SEQN          0
WHD020        0
WHD050        0
WHQ060     5131
WHD080A    4571
WHD080B    5387
WHD080C    5382
WHD080E    5830
WHD080F    6134
WHD080G    6250
WHD080H    6301
WHD080I    6307
WHD080J    6204
WHD080K    6363
WHD080M    4748
WHD080N    6250
WHD080O    5509
WHD080Q    4689
WHD080R    5150
WHD080S    5149
WHD080T    4992
WHD080U    6384
WHQ225        0
WHD110     2263
WHD120     1343
WHD140      335
WHQ150      450
WHQ190        0
WHQ200     6342
dtype: int64

In [148]:
wt_hx_df = wt_hx_df.drop(['WHQ060', 'WHD080A', 'WHD080B', 'WHD080C', 
'WHD080E', 'WHD080F', 'WHD080G', 'WHD080H', 
'WHD080I', 'WHD080J', 'WHD080K',  'WHD080M',
'WHD080N', 'WHD080O',  'WHD080Q',  'WHD080R', 
'WHD080S', 'WHD080T', 'WHD080U', 'WHQ200'], axis=1) 

In [149]:
wt_hx_df.head()

,SEQN,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190
0,83732.0,212,225,2,230.0,205.0,260.0,20.0,2
1,83733.0,193,191,5,180.0,160.0,205.0,51.0,2
2,83734.0,182,212,1,224.0,165.0,240.0,46.0,2
3,83735.0,220,220,2,160.0,120.0,240.0,55.0,2
4,83736.0,125,135,5,127.0,127.0,135.0,33.0,2


In [150]:
wt_hx_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6400 entries, 0 to 6399
Data columns (total 9 columns):
SEQN      6400 non-null float64
WHD020    6400 non-null int64
WHD050    6400 non-null int64
WHQ225    6400 non-null int64
WHD110    4137 non-null float64
WHD120    5057 non-null float64
WHD140    6065 non-null float64
WHQ150    5950 non-null float64
WHQ190    6400 non-null int64
dtypes: float64(5), int64(4)
memory usage: 450.1 KB


# merging

### put df in order to merge
demog_df, med_cond_df, body_measures_df, wt_hx_df, bp_quest_df, 
bp_measure_df, cbc_df, std_chem_df,  
hdl_df, tot_chol_df, hscrp_df,

### mean of nutrients = new col tot_x(nutrient)for diet intake, supp, & total
tot_intake1_df, tot_intake2_df, 
tot_diet_supp_1_df, tot_diet_supp_2_df

In [151]:
nhanes27_df = pd.merge(demog_df, med_cond_df, on=['SEQN'], how='left')

In [152]:
nhanes27_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D
10020,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10021,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10022,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10023,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10024,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
nhanes27_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10025 entries, 0 to 10024
Data columns (total 16 columns):
SEQN        10025 non-null float64
RIAGENDR    10025 non-null int64
RIDAGEYR    9629 non-null float64
WTINT2YR    10025 non-null float64
WTMEC2YR    9598 non-null float64
INDFMPIR    8881 non-null float64
MCQ160B     5744 non-null float64
MCQ160C     5744 non-null float64
MCQ160D     5744 non-null float64
MCQ160E     5744 non-null float64
MCQ300A     5744 non-null float64
MCQ365C     6352 non-null float64
MCQ365D     6352 non-null float64
MCQ370A     6352 non-null float64
MCQ370C     6352 non-null float64
MCQ370D     6352 non-null float64
dtypes: float64(15), int64(1)
memory usage: 1.3 MB


In [154]:
nhanes28_df = pd.merge(nhanes27_df, body_measures_df, on=['SEQN'], how='left')

In [155]:
nhanes28_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD
11615,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9
11616,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9
11617,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9
11618,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9
11619,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9


In [156]:
nhanes28_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11620 entries, 0 to 11619
Data columns (total 23 columns):
SEQN        11620 non-null float64
RIAGENDR    11620 non-null int64
RIDAGEYR    11224 non-null float64
WTINT2YR    11620 non-null float64
WTMEC2YR    11193 non-null float64
INDFMPIR    10476 non-null float64
MCQ160B     5744 non-null float64
MCQ160C     5744 non-null float64
MCQ160D     5744 non-null float64
MCQ160E     5744 non-null float64
MCQ300A     5744 non-null float64
MCQ365C     6352 non-null float64
MCQ365D     6352 non-null float64
MCQ370A     6352 non-null float64
MCQ370C     6352 non-null float64
MCQ370D     6352 non-null float64
BMDSTATS    11193 non-null float64
BMXWT       11094 non-null float64
BMXHT       10418 non-null float64
BMXBMI      10405 non-null float64
BMXARMC     10625 non-null float64
BMXWAIST    9962 non-null float64
BMDAVSAD    8632 non-null float64
dtypes: float64(22), int64(1)
memory usage: 2.1 MB


In [157]:
nhanes29_df = pd.merge(nhanes28_df, wt_hx_df, on=['SEQN'], how='left')

In [158]:
nhanes29_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190
128543,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0
128544,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0
128545,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0
128546,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0
128547,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0


# why am i getting so many duplicate rows on a left/inner merge???

In [159]:
nhanes29_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128548 entries, 0 to 128547
Data columns (total 31 columns):
SEQN        128548 non-null float64
RIAGENDR    128548 non-null int64
RIDAGEYR    128152 non-null float64
WTINT2YR    128548 non-null float64
WTMEC2YR    128121 non-null float64
INDFMPIR    127404 non-null float64
MCQ160B     5744 non-null float64
MCQ160C     5744 non-null float64
MCQ160D     5744 non-null float64
MCQ160E     5744 non-null float64
MCQ300A     5744 non-null float64
MCQ365C     6352 non-null float64
MCQ365D     6352 non-null float64
MCQ370A     6352 non-null float64
MCQ370C     6352 non-null float64
MCQ370D     6352 non-null float64
BMDSTATS    128121 non-null float64
BMXWT       128022 non-null float64
BMXHT       127346 non-null float64
BMXBMI      127333 non-null float64
BMXARMC     127553 non-null float64
BMXWAIST    126890 non-null float64
BMDAVSAD    125560 non-null float64
WHD020      124904 non-null float64
WHD050      124904 non-null float64
WHQ225     

In [160]:
nhanes29_df = nhanes29_df.drop_duplicates()

In [161]:
nhanes29_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190
9967,9.369900e+04,2,6.0,9871.384668,9875.921047,4.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,2.900000e+01,126.2,18.2,20.7,62.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9968,9.370000e+04,1,35.0,42314.288108,43194.215112,NaN,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.000000e+00,7.820000e+01,173.3,26.0,30.6,98.9,21.8,160.0,161.0,5.0,NaN,9999.0,161.0,35.0,2.0
9969,9.370100e+04,1,8.0,42087.657815,48252.504294,4.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,2.880000e+01,126.0,18.1,20.8,62.7,14.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9970,9.370200e+04,2,24.0,107361.906644,105080.445194,3.54,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.000000e+00,5.830000e+01,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0
9996,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0


In [162]:
nhanes30_df = pd.merge(nhanes29_df, bp_quest_df, on=['SEQN'], how='left')

In [163]:
nhanes30_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D
10039,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN
10040,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN
10041,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN
10042,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN
10043,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN


In [164]:
nhanes31_df = pd.merge(nhanes30_df, bp_measure_df, on=['SEQN'], how='left')

In [165]:
nhanes31_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg
14054,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0
14055,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0
14056,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0
14057,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0
14058,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0


In [166]:
nhanes32_df = pd.merge(nhanes31_df, cbc_df, on=['SEQN'], how='left')

In [167]:
nhanes32_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI
153046,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7
153047,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7
153048,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7
153049,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7
153050,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7


In [168]:
nhanes32_df = nhanes32_df.drop_duplicates()

In [169]:
nhanes33_df = pd.merge(nhanes32_df, std_chem_df, on=['SEQN'], how='left')

In [170]:
nhanes33_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA
10022,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2
10023,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2
10024,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2
10025,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2
10026,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2


In [171]:
nhanes34_df = pd.merge(nhanes33_df, hdl_df, on=['SEQN'], how='left')

In [172]:
nhanes34_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD
14390,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40
14391,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40
14392,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40
14393,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40
14394,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40


In [173]:
nhanes35_df = pd.merge(nhanes34_df, tot_chol_df, on=['SEQN'], how='left')

In [174]:
nhanes35_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC
359462,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40
359463,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40
359464,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40
359465,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40
359466,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40


In [175]:
nhanes35_df = nhanes35_df.drop_duplicates()

In [176]:
nhanes36_df = pd.merge(nhanes35_df, hscrp_df, on=['SEQN'], how='left')

In [177]:
nhanes36_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC,LBXHSCRP
10001,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40
10002,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40
10003,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40
10004,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40
10005,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40


In [178]:
nhanes37_df = pd.merge(nhanes36_df, tot_intake1_df, on=['SEQN'], how='left')

In [179]:
nhanes37_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC,LBXHSCRP,WTDRD1,WTDR2D,DRDINT,DBQ095Z,DBD100,DRQSPREP,DR1STY,DRQSDIET,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TMOIS,DR1_320Z,DRD360
11926,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,4.0,0.0,2.0,2.0,1.0,1083.0,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117.0,3.58,131.0,292.0,0.0,1943.0,1.0,6569.0,1828.0,0.57,1.088,11.335,1.269,163.0,0.0,163.0,163.0,192.7,2.18,35.1,1.7,125.1,525.0,830.0,262.0,6.88,5.55,1.175,1397.0,1778.0,66.7,192.0,1982.94,987.0,1.0
11927,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,4.0,0.0,2.0,2.0,1.0,1083.0,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117.0,3.58,131.0,292.0,0.0,1943.0,1.0,6569.0,1828.0,0.57,1.088,11.335,1.269,163.0,0.0,163.0,163.0,192.7,2.18,35.1,1.7,125.1,525.0,830.0,262.0,6.88,5.55,1.175,1397.0,1778.0,66.7,192.0,1982.94,987.0,1.0
11928,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,4.0,0.0,2.0,2.0,1.0,1083.0,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117.0,3.58,131.0,292.0,0.0,1943.0,1.0,6569.0,1828.0,0.57,1.088,11.335,1.269,163.0,0.0,163.0,163.0,192.7,2.18,35.1,1.7,125.1,525.0,830.0,262.0,6.88,5.55,1.175,1397.0,1778.0,66.7,192.0,1982.94,987.0,1.0
11929,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1

In [180]:
nhanes38_df = pd.merge(nhanes37_df, tot_intake2_df, on=['SEQN'], how='left')

In [181]:
nhanes38_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC,LBXHSCRP,WTDRD1_x,WTDR2D_x,DRDINT_x,DBQ095Z,DBD100,DRQSPREP,DR1STY,DRQSDIET,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TMOIS,DR1_320Z,DRD360,WTDRD1_y,WTDR2D_y,DRDINT_y,DR2STY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TMOIS,DR2_320Z
11982,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,4.0,0.0,2.0,2.0,1.0,1083.0,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117.0,3.58,131.0,292.0,0.0,1943.0,1.0,6569.0,1828.0,0.57,1.088,11.335,1.269,163.0,0.0,163.0,163.0,192.7,2.18,35.1,1.7,125.1,525.0,830.0,262.0,6.88,5.55,1.175,1397.0,1778.0,66.7,192.0,1982.94,987.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11983,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,4.0,0.0,2.0,2.0,1.0,1083.0,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117.0,3.58,131.0,292.0,0.0,1943.0,1.0,6569.0,1828.0,0.57,1.088,11.335,1.269,163.0,0.0,163.0,163.0,192.7,2.18,35.1,1.7,125.1,525.0,830.0,262.0,6.88,5.55,1.175,1397.0,1778.0,66.7,192.0,1982.94,987.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11984,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12

In [182]:
nhanes39_df = pd.merge(nhanes38_df, tot_diet_supp_1_df, on=['SEQN'], how='left')

In [183]:
nhanes39_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC,LBXHSCRP,WTDRD1_x,WTDR2D_x,DRDINT_x,DBQ095Z,DBD100,DRQSPREP,DR1STY,DRQSDIET,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TMOIS,DR1_320Z,DRD360,WTDRD1_y,WTDR2D_y,DRDINT_y,DR2STY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TMOIS,DR2_320Z,WTDRD1,WTDR2D,DS1DS,DS1AN,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TVC,DS1TVD,DS1TCALC,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE
119782,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,4.0,0.0,2.0,2.0,1.0,1083.0,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117.0,3.58,131.0,292.0,0.0,1943.0,1.0,6569.0,1828.0,0.57,1.088,11.335,1.269,163.0,0.0,163.0,163.0,192.7,2.18,35.1,1.7,125.1,525.0,830.0,262.0,6.88,5.55,1.175,1397.0,1778.0,66.7,192.0,1982.94,987.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119783,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,4.0,0.0,2.0,2.0,1.0,1083.0,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117.0,3.58,131.0,292.0,0.0,1943.0,1.0,6569.0,1828.0,0.57,1.088,11.335,1.269,163.0,0.0,163.0,163.0,192.7,2.18,35.1,1.7,125.1,525.0,830.0,262.0,6.88,5.55,1.175,1397.0,1778.0,66.7,192.0,1982.94,987.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [184]:
nhanes39_df = nhanes39_df.drop_duplicates()

In [185]:
nhanes40_df = pd.merge(nhanes39_df, tot_diet_supp_2_df, on=['SEQN'], how='left')

In [186]:
nhanes40_df.tail()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC,LBXHSCRP,WTDRD1_x,WTDR2D_x,DRDINT_x,DBQ095Z,DBD100,DRQSPREP,DR1STY,DRQSDIET,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TMOIS,DR1_320Z,DRD360,WTDRD1_y,WTDR2D_y,DRDINT_y,DR2STY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TMOIS,DR2_320Z,WTDRD1_x,WTDR2D_x,DS1DS,DS1AN,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TVC,DS1TVD,DS1TCALC,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE,WTDRD1_y,WTDR2D_y,DS2DS,DS2AN,DS2TVB1,DS2TVB2,DS2TNIAC,DS2TVB6,DS2TVB12,DS2TFA,DS2TFDFE,DS2TVC,DS2TVD,DS2TCALC,DS2TMAGN,DS2TIRON,DS2TZINC,DS2TCOPP,DS2TSODI,DS2TPOTA,DS2TSELE
10022,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,4.0,0.0,2.0,2.0,1.0,1083.0,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117.0,3.58,131.0,292.0,0.0,1943.0,1.0,6569.0,1828.0,0.57,1.088,11.335,1.269,163.0,0.0,163.0,163.0,192.7,2.18,35.1,1.7,125.1,525.0,830.0,262.0,6.88,5.55,1.175,1397.0,1778.0,66.7,192.0,1982.94,987.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.687825e-40,3.687825e-40,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10023,3.687825e-40,2,24.0,107361.906644,105080.445194,3.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.687825e-40,3.687825e-40,165.0,21.4,26.2,72.5,16.9,128.0,128.0,5.0,NaN,NaN,130.0,22.0,2.0,3.687825e-40,3.687825e-40,NaN,NaN,2.0,1.0,1.0,2.0,NaN,118.666667,66.0,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,0.5,0.3,2.4,0.5,6.7,NaN,4.58,12.8,39.4,86.3,28.0,13.8,257.0,8.7,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,25.0,9.0,163.0,78.0,102.0,0.66,101.0,156.0,3.84,93.0,138.0,275.0,2.8,0.904,0.5,6.8,79.0,4.2,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,3.687825e-40,4.0,0.0,2.0,2.0,1.0,1083.0,56.4,70.04,42.7,8.4,42.88,9.798,7.782,21.97,117.0,3.58,131.0,292.0,0.0,1943.0,1.0,6569.0,1828.0,0.57,1.088,11.335,1.269,163.0,0.0,163.0,163.0,192.7,2.18,35.1,1.7,125.1,525.0,830.0,262.0,6.88,5.55,1.175,1397.0,1778.0,66.7

In [187]:
nhanes40_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10027 entries, 0 to 10026
Columns: 226 entries, SEQN to DS2TSELE
dtypes: float64(225), int64(1)
memory usage: 17.4 MB


demog_df['SEQN']   9972;  10,000  +
med_cond_df['SEQN']   9575;  9600  =  10,025  +
body_measures_df['SEQN']  9545;  9600 = 9,600  +
wt_hx_df['SEQN']   6328;  6400  = 9,190  =  9,190  +
bp_measure_df['SEQN']   9545;  9600  = 9,190  +
bp_quest_df['SEQN']   6328;  6400  =  
cbc_df['SEQN']   9166;  9200 = 
std_chem_df['SEQN']   6745;  6800  = 
hdl_df['SEQN']   8022;  8100  = 
tot_chol_df['SEQN']  8022; 8100 = 
hscrp_df['SEQN']   9166;  9200  = 


tot_intake1_df['SEQN']   9545;  9600
tot_intake2_df['SEQN']   9544;  9600

tot_diet_supp_1_df['SEQN']  9545;   9600
tot_diet_supp_2_df['SEQN']  9545;   9600  =  10646


# cleaning data
### reduce each file to relevant data only,  address missing values, convert to numeric, dummy categorical values

In [188]:
nhanes40_df['SEQN'].nunique()

9972

### remove duplicate rows

In [189]:
nhanes40_df = nhanes40_df.drop_duplicates()

In [190]:
nhanes40_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9972 entries, 0 to 9971
Columns: 226 entries, SEQN to DS2TSELE
dtypes: float64(225), int64(1)
memory usage: 17.3 MB


In [191]:
nhanes40_df.isnull().sum()

SEQN             0
RIAGENDR         0
RIDAGEYR       396
WTINT2YR         0
WTMEC2YR       427
INDFMPIR      1144
MCQ160B       4253
MCQ160C       4253
MCQ160D       4253
MCQ160E       4253
MCQ300A       4253
MCQ365C       3645
MCQ365D       3645
MCQ370A       3645
MCQ370C       3645
MCQ370D       3645
BMDSTATS       427
BMXWT          526
BMXHT         1202
BMXBMI        1215
BMXARMC        995
BMXWAIST      1658
BMDAVSAD      2988
WHD020        3644
WHD050        3644
WHQ225        3644
WHD110        5835
WHD120        4915
WHD140        3979
WHQ150        4094
WHQ190        3644
BPQ020        3644
BPQ030        7881
BPQ040A       7882
BPQ050A       8121
BPQ080        3644
BPQ060        5604
BPQ070        5333
BPQ090D       5333
BPQ100D       8448
bp_sys_avg    2608
bp_dis_avg    2636
LBXWBCSI      1854
LBXLYPCT      1855
LBXMOPCT      1855
LBXNEPCT      1855
LBXEOPCT      1872
LBXBAPCT      1855
LBDLYMNO      1855
LBDMONO       1855
LBDNENO       1855
LBDEONO       2267
LBXRBCSI    

### for NaN in medical condition, wt hx,  bp quest, dietary intake 1&2 quest
change all missing values to: 9 = don't know

In [193]:
fill_list = ['MCQ160B', 'MCQ160C', 'MCQ160D', 'MCQ160E', 'MCQ300A',
             'MCQ365C', 'MCQ365D', 'MCQ370A', 'MCQ370C', 'MCQ370D',  
             'WHD020', 'WHD050', 'WHQ225', 'WHD110', 'WHD120', 
             'WHD140', 'WHQ150', 'WHQ190', 'BPQ020', 'BPQ030', 
             'BPQ040A', 'BPQ050A', 'BPQ080', 'BPQ060', 'BPQ070',  
             'BPQ090D', 'BPQ100D', 'DBD100', 'DRQSPREP', 'DR1STY', 
             'DRQSDIET', 'DR2STY', 'DS1DS', 'DS1AN', 'DS2DS', 'DS2AN'] 

In [194]:
for column in fill_list:
    nhanes40_df.loc[:, column] = nhanes40_df.loc[:, column].fillna(9)

In [195]:
nhanes40_df.isnull().sum()

SEQN             0
RIAGENDR         0
RIDAGEYR       396
WTINT2YR         0
WTMEC2YR       427
INDFMPIR      1144
MCQ160B          0
MCQ160C          0
MCQ160D          0
MCQ160E          0
MCQ300A          0
MCQ365C          0
MCQ365D          0
MCQ370A          0
MCQ370C          0
MCQ370D          0
BMDSTATS       427
BMXWT          526
BMXHT         1202
BMXBMI        1215
BMXARMC        995
BMXWAIST      1658
BMDAVSAD      2988
WHD020           0
WHD050           0
WHQ225           0
WHD110           0
WHD120           0
WHD140           0
WHQ150           0
WHQ190           0
BPQ020           0
BPQ030           0
BPQ040A          0
BPQ050A          0
BPQ080           0
BPQ060           0
BPQ070           0
BPQ090D          0
BPQ100D          0
bp_sys_avg    2608
bp_dis_avg    2636
LBXWBCSI      1854
LBXLYPCT      1855
LBXMOPCT      1855
LBXNEPCT      1855
LBXEOPCT      1872
LBXBAPCT      1855
LBDLYMNO      1855
LBDMONO       1855
LBDNENO       1855
LBDEONO       2267
LBXRBCSI    

In [199]:
nhanes40_df.head()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC,LBXHSCRP,WTDRD1_x,WTDR2D_x,DRDINT_x,DBQ095Z,DBD100,DRQSPREP,DR1STY,DRQSDIET,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TMOIS,DR1_320Z,DRD360,WTDRD1_y,WTDR2D_y,DRDINT_y,DR2STY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TMOIS,DR2_320Z,WTDRD1_x,WTDR2D_x,DS1DS,DS1AN,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TVC,DS1TVD,DS1TCALC,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE,WTDRD1_y,WTDR2D_y,DS2DS,DS2AN,DS2TVB1,DS2TVB2,DS2TNIAC,DS2TVB6,DS2TVB12,DS2TFA,DS2TFDFE,DS2TVC,DS2TVD,DS2TCALC,DS2TMAGN,DS2TIRON,DS2TZINC,DS2TCOPP,DS2TSODI,DS2TPOTA,DS2TSELE
0,83732.0,1,62.0,134671.370419,135629.507405,4.39,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,94.8,184.5,27.8,35.9,101.1,22.8,212.0,225.0,2.0,230.0,205.0,260.0,20.0,2.0,2.0,9.0,9.0,9.0,2.0,1.0,1.0,2.0,9.0,122.666667,65.333333,9.8,23.9,8.2,63.5,4.0,0.5,2.3,0.8,6.2,0.4,4.93,15.2,44.7,90.8,30.8,13.9,181.0,8.3,4.6,52.0,21.0,25.0,13.0,25.0,9.8,172.0,203.0,100.0,0.90,94.0,70.0,4.30,110.0,136.0,272.0,4.7,1.518,0.5,7.5,158.0,4.2,46.0,173.0,0.6,92670.699919,69945.934107,2.0,1.0,3.0,3.0,1.0,2.0,1781.0,76.03,193.29,42.31,23.6,79.24,23.430,31.897,18.528,138.0,10.59,230.0,307.0,336.0,624.0,282.0,1206.0,1120.0,2.344,1.949,20.401,2.279,450.0,163.0,286.0,564.0,290.7,1.89,110.5,2.6,139.0,623.0,1052.0,255.0,16.01,8.73,1.095,5298.0,2641.0,113.6,360.0,3028.29,960.0,1.0,92670.699919,69945.934107,2.0,1.0,2744.0,133.60,281.64,118.40,15.9,121.59,40.420,42.428,26.813,635.0,8.62,494.0,996.0,91.0,5970.0,42.0,1715.0,8474.0,2.027,3.286,35.661,3.283,395.0,160.0,235.0,507.0,540.4,7.01,64.6,9.6,533.4,1433.0,1880.0,334.0,15.67,15.09,1.265,4431.0,3774.0,167.8,3694.92,1440.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,4.2,11.0,0.5,30.0,290.0,19.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,2.1,11.0,0.5,15.0,185.0,19.0
1,83733.0,1,53.0,24328.560239,25282.425927,1.32,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,90.4,171.4,30.8,33.2,107.9,27.3,193.0,191.0,5.0,180.0,160.0,205.0,51.0,2.0,2.0,9.0,9.0,9.0,2.0,2.0,9.0,9.0,9.0,140.000000,86.000000,7.3,31.3,9.7,54.8,2.6,1.8,2.3,0.7,4.0,0.2,4.89,17.5,49.7,101.8,35.8,13.4,170.0,9.6,4.5,47.0,31.0,35.0,10.0,27.0,9.8,267.0,91.0,103.0,1.05,94.0,161.0,4.55,108.0,140.0,278.0,4.4,1.421,0.6,7.4,170.0,7.0,63.0,265.0,1.4,16454.105994,13190.268870,2.0,4.0,0.0,2.0,2.0,2.0,2964.0,62.36,356.85,180.84,7.3,77.91,25.722,19.098,19.216,407.0,4.19,352.0,453.0,405.0,966.0,20.0,0.0,1279.0,1.859,2.478,29.444,1.823,478.0,112.0,365.0,556.0,672.7,2.92,30.2,4.6,87.6,594.0,1414.0,262.0,11.01,4.82,0.571,3164.0,2249.0,89.9,192.0,4024.25,507.0,1.0,16454.105994,13190.268870,2.0,2.0,3971.0,165.86,223.57,34.52,12.2,154.55,32.969,61.

In [210]:
nhanes40_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9544 entries, 0 to 9970
Columns: 226 entries, SEQN to DS2TSELE
dtypes: float64(225), int64(1)
memory usage: 16.5 MB


In [215]:
#creating a df for diet intake review
nhanes_diet_df = nhanes40_df.dropna(subset=['DR2TKCAL'])

In [217]:
nhanes_diet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9544 entries, 0 to 9970
Columns: 226 entries, SEQN to DS2TSELE
dtypes: float64(225), int64(1)
memory usage: 16.5 MB


In [218]:
nhanes_diet_df.isnull().sum()

SEQN             0
RIAGENDR         0
RIDAGEYR       379
WTINT2YR         0
WTMEC2YR         0
INDFMPIR      1025
MCQ160B          0
MCQ160C          0
MCQ160D          0
MCQ160E          0
MCQ300A          0
MCQ365C          0
MCQ365D          0
MCQ370A          0
MCQ370C          0
MCQ370D          0
BMDSTATS         0
BMXWT           99
BMXHT          775
BMXBMI         788
BMXARMC        568
BMXWAIST      1231
BMDAVSAD      2561
WHD020           0
WHD050           0
WHQ225           0
WHD110           0
WHD120           0
WHD140           0
WHQ150           0
WHQ190           0
BPQ020           0
BPQ030           0
BPQ040A          0
BPQ050A          0
BPQ080           0
BPQ060           0
BPQ070           0
BPQ090D          0
BPQ100D          0
bp_sys_avg    2181
bp_dis_avg    2209
LBXWBCSI      1427
LBXLYPCT      1428
LBXMOPCT      1428
LBXNEPCT      1428
LBXEOPCT      1445
LBXBAPCT      1428
LBDLYMNO      1428
LBDMONO       1428
LBDNENO       1428
LBDEONO       1839
LBXRBCSI    

In [219]:
list(nhanes_diet_df.columns)

['SEQN',
 'RIAGENDR',
 'RIDAGEYR',
 'WTINT2YR',
 'WTMEC2YR',
 'INDFMPIR',
 'MCQ160B',
 'MCQ160C',
 'MCQ160D',
 'MCQ160E',
 'MCQ300A',
 'MCQ365C',
 'MCQ365D',
 'MCQ370A',
 'MCQ370C',
 'MCQ370D',
 'BMDSTATS',
 'BMXWT',
 'BMXHT',
 'BMXBMI',
 'BMXARMC',
 'BMXWAIST',
 'BMDAVSAD',
 'WHD020',
 'WHD050',
 'WHQ225',
 'WHD110',
 'WHD120',
 'WHD140',
 'WHQ150',
 'WHQ190',
 'BPQ020',
 'BPQ030',
 'BPQ040A',
 'BPQ050A',
 'BPQ080',
 'BPQ060',
 'BPQ070',
 'BPQ090D',
 'BPQ100D',
 'bp_sys_avg',
 'bp_dis_avg',
 'LBXWBCSI',
 'LBXLYPCT',
 'LBXMOPCT',
 'LBXNEPCT',
 'LBXEOPCT',
 'LBXBAPCT',
 'LBDLYMNO',
 'LBDMONO',
 'LBDNENO',
 'LBDEONO',
 'LBXRBCSI',
 'LBXHGB',
 'LBXHCT',
 'LBXMCVSI',
 'LBXMCHSI',
 'LBXRDW',
 'LBXPLTSI',
 'LBXMPSI',
 'LBXSAL',
 'LBXSAPSI',
 'LBXSASSI',
 'LBXSATSI',
 'LBXSBU',
 'LBXSC3SI',
 'LBXSCA',
 'LBXSCH',
 'LBXSCK',
 'LBXSCLSI',
 'LBXSCR',
 'LBXSGL',
 'LBXSIR',
 'LBXSKSI',
 'LBXSLDSI',
 'LBXSNASI',
 'LBXSOSSI',
 'LBXSPH',
 'LBDSPHSI',
 'LBXSTB',
 'LBXSTP',
 'LBXSTR',
 'LBXSUA',
 'LBDHD

## compute average intake: total, diet, supplement

#### manual computation of average intake

In [226]:
nhanes_diet_df['kcal_avg'] = nhanes_diet_df[['DR1TKCAL', 'DR2TKCAL']].mean(axis=1)

In [239]:
nhanes_diet_df.head()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC,LBXHSCRP,WTDRD1_x,WTDR2D_x,DRDINT_x,DBQ095Z,DBD100,DRQSPREP,DR1STY,DRQSDIET,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TMOIS,DR1_320Z,DRD360,WTDRD1_y,WTDR2D_y,DRDINT_y,DR2STY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TMOIS,DR2_320Z,WTDRD1_x,WTDR2D_x,DS1DS,DS1AN,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TVC,DS1TVD,DS1TCALC,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE,WTDRD1_y,WTDR2D_y,DS2DS,DS2AN,DS2TVB1,DS2TVB2,DS2TNIAC,DS2TVB6,DS2TVB12,DS2TFA,DS2TFDFE,DS2TVC,DS2TVD,DS2TCALC,DS2TMAGN,DS2TIRON,DS2TZINC,DS2TCOPP,DS2TSODI,DS2TPOTA,DS2TSELE,diet_avg,kcal_avg,pro_avg,carb_avg,sugar_avg,fiber_avg,fat_avg,sfat_avg,mfat_avg,pfat_avg,chol_avg
0,83732.0,1,62.0,134671.370419,135629.507405,4.39,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,94.8,184.5,27.8,35.9,101.1,22.8,212.0,225.0,2.0,230.0,205.0,260.0,20.0,2.0,2.0,9.0,9.0,9.0,2.0,1.0,1.0,2.0,9.0,122.666667,65.333333,9.8,23.9,8.2,63.5,4.0,0.5,2.3,0.8,6.2,0.4,4.93,15.2,44.7,90.8,30.8,13.9,181.0,8.3,4.6,52.0,21.0,25.0,13.0,25.0,9.8,172.0,203.0,100.0,0.90,94.0,70.0,4.30,110.0,136.0,272.0,4.7,1.518,0.5,7.5,158.0,4.2,46.0,173.0,0.6,92670.699919,69945.934107,2.0,1.0,3.0,3.0,1.0,2.0,1781.0,76.03,193.29,42.31,23.6,79.24,23.430,31.897,18.528,138.0,10.59,230.0,307.0,336.0,624.0,282.0,1206.0,1120.0,2.344,1.949,20.401,2.279,450.0,163.0,286.0,564.0,290.7,1.89,110.5,2.6,139.0,623.0,1052.0,255.0,16.01,8.73,1.095,5298.0,2641.0,113.6,360.0,3028.29,960.0,1.0,92670.699919,69945.934107,2.0,1.0,2744.0,133.60,281.64,118.40,15.9,121.59,40.420,42.428,26.813,635.0,8.62,494.0,996.0,91.0,5970.0,42.0,1715.0,8474.0,2.027,3.286,35.661,3.283,395.0,160.0,235.0,507.0,540.4,7.01,64.6,9.6,533.4,1433.0,1880.0,334.0,15.67,15.09,1.265,4431.0,3774.0,167.8,3694.92,1440.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,4.2,11.0,0.5,30.0,290.0,19.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,2.1,11.0,0.5,15.0,185.0,19.0,386.5,2262.5,104.815,386.5,386.5,386.5,386.5,386.5,386.5,386.5,386.5
1,83733.0,1,53.0,24328.560239,25282.425927,1.32,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,90.4,171.4,30.8,33.2,107.9,27.3,193.0,191.0,5.0,180.0,160.0,205.0,51.0,2.0,2.0,9.0,9.0,9.0,2.0,2.0,9.0,9.0,9.0,140.000000,86.000000,7.3,31.3,9.7,54.8,2.6,1.8,2.3,0.7,4.0,0.2,4.89,17.5,49.7,101.8,35.8,13.4,170.0,9.6,4.5,47.0,31.0,35.0,10.0,27.0,9.8,267.0,91.0,103.0,1.05,94.0,161.0,4.55,108.0,140.0,278.0,4.4,1.421,0.6,7.4,170.0,7.0,63.0,265.0,1.4,16454.105994,13190.268870,2.0,4.0,0.0,2.0,2.0,2.0,2964.0,62.36,356.85,180.84,7.3,77.91,25.722,19.098,19.216,407.0,4.19,352.0,453.0,405.0,966.0,20.0,0.0,1279.0,1.859,2.478,29.444,1.823,478.0,112.0,365.0,556.0,672.7,2.92,30.2,4.6

In [238]:
nhanes_diet_df['pro_avg'] = nhanes_diet_df[['DR1TPROT', 'DR2TPROT']].mean(axis=1)

In [240]:
nhanes_diet_df['carb_avg'] = nhanes_diet_df[['DR1TCARB', 'DR2TCARB']].mean(axis=1)

In [241]:
nhanes_diet_df['sugar_avg'] = nhanes_diet_df[['DR1TSUGR', 'DR2TSUGR']].mean(axis=1)

In [243]:
nhanes_diet_df['fiber_avg'] = nhanes_diet_df[['DR1TFIBE', 'DR2TFIBE']].mean(axis=1)

In [244]:
nhanes_diet_df['fat_avg'] = nhanes_diet_df[['DR1TTFAT', 'DR2TTFAT']].mean(axis=1)

In [245]:
nhanes_diet_df['sfat_avg'] = nhanes_diet_df[['DR1TSFAT', 'DR2TSFAT']].mean(axis=1)

In [246]:
nhanes_diet_df['mfat_avg'] = nhanes_diet_df[['DR1TMFAT', 'DR2TMFAT']].mean(axis=1)

In [247]:
nhanes_diet_df['pfat_avg'] = nhanes_diet_df[['DR1TPFAT', 'DR2TPFAT']].mean(axis=1)

In [248]:
nhanes_diet_df['chol_avg'] = nhanes_diet_df[['DR1TCHOL', 'DR2TCHOL']].mean(axis=1)

In [249]:
nhanes_diet_df.head()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC,LBXHSCRP,WTDRD1_x,WTDR2D_x,DRDINT_x,DBQ095Z,DBD100,DRQSPREP,DR1STY,DRQSDIET,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TMOIS,DR1_320Z,DRD360,WTDRD1_y,WTDR2D_y,DRDINT_y,DR2STY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TMOIS,DR2_320Z,WTDRD1_x,WTDR2D_x,DS1DS,DS1AN,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TVC,DS1TVD,DS1TCALC,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE,WTDRD1_y,WTDR2D_y,DS2DS,DS2AN,DS2TVB1,DS2TVB2,DS2TNIAC,DS2TVB6,DS2TVB12,DS2TFA,DS2TFDFE,DS2TVC,DS2TVD,DS2TCALC,DS2TMAGN,DS2TIRON,DS2TZINC,DS2TCOPP,DS2TSODI,DS2TPOTA,DS2TSELE,diet_avg,kcal_avg,pro_avg,carb_avg,sugar_avg,fiber_avg,fat_avg,sfat_avg,mfat_avg,pfat_avg,chol_avg
0,83732.0,1,62.0,134671.370419,135629.507405,4.39,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,94.8,184.5,27.8,35.9,101.1,22.8,212.0,225.0,2.0,230.0,205.0,260.0,20.0,2.0,2.0,9.0,9.0,9.0,2.0,1.0,1.0,2.0,9.0,122.666667,65.333333,9.8,23.9,8.2,63.5,4.0,0.5,2.3,0.8,6.2,0.4,4.93,15.2,44.7,90.8,30.8,13.9,181.0,8.3,4.6,52.0,21.0,25.0,13.0,25.0,9.8,172.0,203.0,100.0,0.90,94.0,70.0,4.30,110.0,136.0,272.0,4.7,1.518,0.5,7.5,158.0,4.2,46.0,173.0,0.6,92670.699919,69945.934107,2.0,1.0,3.0,3.0,1.0,2.0,1781.0,76.03,193.29,42.31,23.6,79.24,23.430,31.897,18.528,138.0,10.59,230.0,307.0,336.0,624.0,282.0,1206.0,1120.0,2.344,1.949,20.401,2.279,450.0,163.0,286.0,564.0,290.7,1.89,110.5,2.6,139.0,623.0,1052.0,255.0,16.01,8.73,1.095,5298.0,2641.0,113.6,360.0,3028.29,960.0,1.0,92670.699919,69945.934107,2.0,1.0,2744.0,133.60,281.64,118.40,15.9,121.59,40.420,42.428,26.813,635.0,8.62,494.0,996.0,91.0,5970.0,42.0,1715.0,8474.0,2.027,3.286,35.661,3.283,395.0,160.0,235.0,507.0,540.4,7.01,64.6,9.6,533.4,1433.0,1880.0,334.0,15.67,15.09,1.265,4431.0,3774.0,167.8,3694.92,1440.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,4.2,11.0,0.5,30.0,290.0,19.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,2.1,11.0,0.5,15.0,185.0,19.0,386.5,2262.5,104.815,237.465,80.355,19.75,100.415,31.9250,37.1625,22.6705,386.5
1,83733.0,1,53.0,24328.560239,25282.425927,1.32,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,90.4,171.4,30.8,33.2,107.9,27.3,193.0,191.0,5.0,180.0,160.0,205.0,51.0,2.0,2.0,9.0,9.0,9.0,2.0,2.0,9.0,9.0,9.0,140.000000,86.000000,7.3,31.3,9.7,54.8,2.6,1.8,2.3,0.7,4.0,0.2,4.89,17.5,49.7,101.8,35.8,13.4,170.0,9.6,4.5,47.0,31.0,35.0,10.0,27.0,9.8,267.0,91.0,103.0,1.05,94.0,161.0,4.55,108.0,140.0,278.0,4.4,1.421,0.6,7.4,170.0,7.0,63.0,265.0,1.4,16454.105994,13190.268870,2.0,4.0,0.0,2.0,2.0,2.0,2964.0,62.36,356.85,180.84,7.3,77.91,25.722,19.098,19.216,407.0,4.19,352.0,453.0,405.0,966.0,20.0,0.0,1279.0,1.859,2.478,29.444,1.823,478.0,112.0,365.0,556.0,672.7,2.

# begin attempts

#### need to write code to iterate through 3 lists: avg_columns (new column names), macro1 & macro2 so as to create a new column name for the MEAN of macro1 & macro2 across rows

In [234]:
avg_columns = ['kcal_avg','pro_avg', 'carb_avg', 'sugar_avg', 'fiber_avg', 
               'fat_avg', 'sfat_avg', 'mfat_avg', 'pfat_avg', 'chol_avg']

In [235]:
macro1_nutrient_list = ['DR1TKCAL',  'DR1TPROT', 'DR1TCARB', 'DR1TSUGR', 
                        'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 
                        'DR1TPFAT', 'DR1TCHOL']
macro2_nutrient_list = ['DR2TKCAL', 'DR2TPROT', 'DR2TCARB', 'DR2TSUGR',
                        'DR2TFIBE', 'DR2TTFAT', 'DR2TSFAT', 'DR2TMFAT',
                        'DR2TPFAT', 'DR2TCHOL']

In [237]:
for c, a, b in enumerate(avg_columns):
    nhanes_diet_df[avg_columns[c]] = nhanes_diet_df[[macro1_nutrient_list[a], macro2_nutrient_list[b]]].mean(axis=1)

ValueError: not enough values to unpack (expected 3, got 2)

In [270]:
micro_avg_list = ['vit_e_avg', 'retinol_avg', 'vit_a_avg', 'alpha_c_avg',
                  'beta_c_avg', 'beta_cry_avg', 'lyco_avg', 'lutein_avg',
                  'vit_b1_avg', 'vit_b2_avg', 'niac_avg', 'vit_b6_avg',
                  'folate_avg', 'folic_acid_avg', 'food_fol_avg', 
                  'folate_dfe_avg', 'chol_avg', 'vit_b12_avg', 'vit_c_avg',
                  'vit_d_avg', 'vit_k_avg', 'calc_avg', 'phos_avg', 
                  'mag_avg', 'fe_avg', 'zn_avg', 'cu_avg', 'na_avg',
                  'k_avg', 'se_avg', 'mois_avg', 'h20_avg']

micro1_nutrient_list = ['DR1TATOC', 'DR1TRET', 'DR1TVARA', 'DR1TACAR',
                        'DR1TBCAR', 'DR1TCRYP', 'DR1TLYCO', 'DR1TLZ',
                        'DR1TVB1', 'DR1TVB2', 'DR1TNIAC', 'DR1TVB6', 
                        'DR1TFOLA', 'DR1TFA', 'DR1TFF', 'DR1TFDFE', 
                        'DR1TCHL', 'DR1TVB12', 'DR1TVC', 'DR1TVD', 
                        'DR1TVK', 'DR1TCALC', 'DR1TPHOS', 'DR1TMAGN', 
                        'DR1TIRON', 'DR1TZINC', 'DR1TCOPP', 'DR1TSODI', 
                        'DR1TPOTA', 'DR1TSELE', 'DR1TMOIS', 'DR1_320Z']

micro2_nutrient_list = ['DR2TATOC', 'DR2TRET', 'DR2TVARA', 'DR2TACAR', 
                        'DR2TBCAR', 'DR2TCRYP', 'DR2TLYCO', 'DR2TLZ',
                        'DR2TVB1', 'DR2TVB2', 'DR2TNIAC', 'DR2TVB6',
                        'DR2TFOLA', 'DR2TFA', 'DR2TFF', 'DR2TFDFE',
                        'DR2TCHL', 'DR2TVB12', 'DR2TVC', 'DR2TVD',
                        'DR2TVK', 'DR2TCALC', 'DR2TPHOS', 'DR2TMAGN',
                        'DR2TIRON', 'DR2TZINC', 'DR2TCOPP', 'DR2TSODI',
                        'DR2TPOTA', 'DR2TSELE', 'DR2TMOIS', 'DR2_320Z']

## attempt 1

In [275]:
#this does not give correct mean for each row
for c in micro_avg_list: # for each new column name
    for i in range(len(micro1_nutrient_list)):
         nhanes_diet_df[c] = nhanes_diet_df[[micro1_nutrient_list[i], macro2_nutrient_list[i]]].mean(axis=1)


IndexError: list index out of range

In [272]:
nhanes_diet_df.head()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC,LBXHSCRP,WTDRD1_x,WTDR2D_x,DRDINT_x,DBQ095Z,DBD100,DRQSPREP,DR1STY,DRQSDIET,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TMOIS,DR1_320Z,DRD360,WTDRD1_y,WTDR2D_y,DRDINT_y,DR2STY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TMOIS,DR2_320Z,WTDRD1_x,WTDR2D_x,DS1DS,DS1AN,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TVC,DS1TVD,DS1TCALC,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE,WTDRD1_y,WTDR2D_y,DS2DS,DS2AN,DS2TVB1,DS2TVB2,DS2TNIAC,DS2TVB6,DS2TVB12,DS2TFA,DS2TFDFE,DS2TVC,DS2TVD,DS2TCALC,DS2TMAGN,DS2TIRON,DS2TZINC,DS2TCOPP,DS2TSODI,DS2TPOTA,DS2TSELE,diet_avg,kcal_avg,pro_avg,carb_avg,sugar_avg,fiber_avg,fat_avg,sfat_avg,mfat_avg,pfat_avg,chol_avg,vit_e_avg,retinol_avg,vit_a_avg,alpha_c_avg,beta_c_avg,beta_cry_avg,lyco_avg,lutein_avg,vit_b1_avg,vit_b2_avg,niac_avg,vit_b6_avg,folate_avg,folic_acid_avg,food_fol_avg,folate_dfe_avg,vit_b12_avg,vit_c_avg,vit_d_avg,vit_k_avg,calc_avg,phos_avg,mag_avg,fe_avg,zn_avg,cu_avg,na_avg,k_avg,se_avg,mois_avg,h20_avg
0,83732.0,1,62.0,134671.370419,135629.507405,4.39,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,94.8,184.5,27.8,35.9,101.1,22.8,212.0,225.0,2.0,230.0,205.0,260.0,20.0,2.0,2.0,9.0,9.0,9.0,2.0,1.0,1.0,2.0,9.0,122.666667,65.333333,9.8,23.9,8.2,63.5,4.0,0.5,2.3,0.8,6.2,0.4,4.93,15.2,44.7,90.8,30.8,13.9,181.0,8.3,4.6,52.0,21.0,25.0,13.0,25.0,9.8,172.0,203.0,100.0,0.90,94.0,70.0,4.30,110.0,136.0,272.0,4.7,1.518,0.5,7.5,158.0,4.2,46.0,173.0,0.6,92670.699919,69945.934107,2.0,1.0,3.0,3.0,1.0,2.0,1781.0,76.03,193.29,42.31,23.6,79.24,23.430,31.897,18.528,138.0,10.59,230.0,307.0,336.0,624.0,282.0,1206.0,1120.0,2.344,1.949,20.401,2.279,450.0,163.0,286.0,564.0,290.7,1.89,110.5,2.6,139.0,623.0,1052.0,255.0,16.01,8.73,1.095,5298.0,2641.0,113.6,360.0,3028.29,960.0,1.0,92670.699919,69945.934107,2.0,1.0,2744.0,133.60,281.64,118.40,15.9,121.59,40.420,42.428,26.813,635.0,8.62,494.0,996.0,91.0,5970.0,42.0,1715.0,8474.0,2.027,3.286,35.661,3.283,395.0,160.0,235.0,507.0,540.4,7.01,64.6,9.6,533.4,1433.0,1880.0,334.0,15.67,15.09,1.265,4431.0,3774.0,167.8,3694.92,1440.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,4.2,11.0,0.5,30.0,290.0,19.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,2.1,11.0,0.5,15.0,185.0,19.0,386.5,2262.5,104.815,237.465,80.355,19.75,100.415,31.9250,37.1625,22.6705,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605
1,83733.0,1,53.0,24328.560239,25282.425927,1.32,2.0,2.0,2.0,2.0

### 'vit_e_avg' should be 9.605; currently = 546.800
[micro1_nutrient_list[i] = 'DR1TATOC' = 10.59
macro2_nutrient_list[1]] = 'DR2TATOC' = 8.62

## attempt 2

In [276]:
for c in micro_avg_list: # for each new column name
    for i in range(len(micro1_nutrient_list)):
         nhanes_diet_df[c] = nhanes_diet_df[micro1_nutrient_list[i]] + nhanes_diet_df[macro2_nutrient_list[i]]/2


IndexError: list index out of range

## attempt 3

In [277]:
## attempt 3 gives INdex error
for c in micro_avg_list: # for each new column name
    for i in range(len(micro1_nutrient_list)):
        for i in range(len(micro2_nutrient_list)):
            nhanes_diet_df[c] = nhanes_diet_df[[micro1_nutrient_list[i], macro2_nutrient_list[i]]].mean(axis=1)


IndexError: list index out of range

## attempt 4

In [278]:
## attempt 4 gives Type Error
for c in micro_avg_list:
    for i in range(len(micro1_nutrient_list, micro2_nutrient_list)):
        nhanes_diet_df[c] = nhanes_diet_df[[micro1_nutrient_list[i], macro2_nutrient_list[i]]].mean(axis=1)


TypeError: len() takes exactly one argument (2 given)

## attempt 5

In [271]:
## attempt 5 gives INCORRECT DATA
for c in micro_avg_list:
    for i in range(len(micro1_nutrient_list)-1):
        nhanes_diet_df[c] = nhanes_diet_df[[micro1_nutrient_list[i], micro2_nutrient_list[i]]].mean(axis=1)

In [274]:
nhanes_diet_df.head()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC,LBXHSCRP,WTDRD1_x,WTDR2D_x,DRDINT_x,DBQ095Z,DBD100,DRQSPREP,DR1STY,DRQSDIET,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TMOIS,DR1_320Z,DRD360,WTDRD1_y,WTDR2D_y,DRDINT_y,DR2STY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TMOIS,DR2_320Z,WTDRD1_x,WTDR2D_x,DS1DS,DS1AN,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TVC,DS1TVD,DS1TCALC,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE,WTDRD1_y,WTDR2D_y,DS2DS,DS2AN,DS2TVB1,DS2TVB2,DS2TNIAC,DS2TVB6,DS2TVB12,DS2TFA,DS2TFDFE,DS2TVC,DS2TVD,DS2TCALC,DS2TMAGN,DS2TIRON,DS2TZINC,DS2TCOPP,DS2TSODI,DS2TPOTA,DS2TSELE,diet_avg,kcal_avg,pro_avg,carb_avg,sugar_avg,fiber_avg,fat_avg,sfat_avg,mfat_avg,pfat_avg,chol_avg,vit_e_avg,retinol_avg,vit_a_avg,alpha_c_avg,beta_c_avg,beta_cry_avg,lyco_avg,lutein_avg,vit_b1_avg,vit_b2_avg,niac_avg,vit_b6_avg,folate_avg,folic_acid_avg,food_fol_avg,folate_dfe_avg,vit_b12_avg,vit_c_avg,vit_d_avg,vit_k_avg,calc_avg,phos_avg,mag_avg,fe_avg,zn_avg,cu_avg,na_avg,k_avg,se_avg,mois_avg,h20_avg
0,83732.0,1,62.0,134671.370419,135629.507405,4.39,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,94.8,184.5,27.8,35.9,101.1,22.8,212.0,225.0,2.0,230.0,205.0,260.0,20.0,2.0,2.0,9.0,9.0,9.0,2.0,1.0,1.0,2.0,9.0,122.666667,65.333333,9.8,23.9,8.2,63.5,4.0,0.5,2.3,0.8,6.2,0.4,4.93,15.2,44.7,90.8,30.8,13.9,181.0,8.3,4.6,52.0,21.0,25.0,13.0,25.0,9.8,172.0,203.0,100.0,0.90,94.0,70.0,4.30,110.0,136.0,272.0,4.7,1.518,0.5,7.5,158.0,4.2,46.0,173.0,0.6,92670.699919,69945.934107,2.0,1.0,3.0,3.0,1.0,2.0,1781.0,76.03,193.29,42.31,23.6,79.24,23.430,31.897,18.528,138.0,10.59,230.0,307.0,336.0,624.0,282.0,1206.0,1120.0,2.344,1.949,20.401,2.279,450.0,163.0,286.0,564.0,290.7,1.89,110.5,2.6,139.0,623.0,1052.0,255.0,16.01,8.73,1.095,5298.0,2641.0,113.6,360.0,3028.29,960.0,1.0,92670.699919,69945.934107,2.0,1.0,2744.0,133.60,281.64,118.40,15.9,121.59,40.420,42.428,26.813,635.0,8.62,494.0,996.0,91.0,5970.0,42.0,1715.0,8474.0,2.027,3.286,35.661,3.283,395.0,160.0,235.0,507.0,540.4,7.01,64.6,9.6,533.4,1433.0,1880.0,334.0,15.67,15.09,1.265,4431.0,3774.0,167.8,3694.92,1440.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,4.2,11.0,0.5,30.0,290.0,19.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,2.1,11.0,0.5,15.0,185.0,19.0,386.5,2262.5,104.815,237.465,80.355,19.75,100.415,31.9250,37.1625,22.6705,3361.605,9.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605,3361.605
1,83733.0,1,53.0,24328.560239,25282.425927,1.32,2.0,2.0,2.0,2.0,2.

### 'vit_e_avg' should be 9.605; currently = 1026.800
[micro1_nutrient_list[i] = 'DR1TATOC' = 10.59
macro2_nutrient_list[1]] = 'DR2TATOC' = 8.62

## end attempts

#### manual calculation of mean for micro1 & micro2
still waiting for help from Slack

In [ ]:
micro_avg_list = ['vit_e_avg', 'retinol_avg', 'vit_a_avg', 'alpha_c_avg',
                  'beta_c_avg', 'beta_cry_avg', 'lyco_avg', 'lutein_avg',
                  'vit_b1_avg', 'vit_b2_avg', 'niac_avg', 'vit_b6_avg',
                  'folate_avg', 'folic_acid_avg', 'food_fol_avg', 
                  'folate_dfe_avg', 'chol_avg', 'vit_b12_avg', 'vit_c_avg',
                  'vit_d_avg', 'vit_k_avg', 'calc_avg', 'phos_avg', 
                  'mag_avg', 'fe_avg', 'zn_avg', 'cu_avg', 'na_avg',
                  'k_avg', 'se_avg', 'mois_avg', 'h20_avg']

micro1_nutrient_list = ['DR1TATOC', 'DR1TRET', 'DR1TVARA', 'DR1TACAR',
                        'DR1TBCAR', 'DR1TCRYP', 'DR1TLYCO', 'DR1TLZ',
                        'DR1TVB1', 'DR1TVB2', 'DR1TNIAC', 'DR1TVB6', 
                        'DR1TFOLA', 'DR1TFA', 'DR1TFF', 'DR1TFDFE', 
                        'DR1TCHL', 'DR1TVB12', 'DR1TVC', 'DR1TVD', 
                        'DR1TVK', 'DR1TCALC', 'DR1TPHOS', 'DR1TMAGN', 
                        'DR1TIRON', 'DR1TZINC', 'DR1TCOPP', 'DR1TSODI', 
                        'DR1TPOTA', 'DR1TSELE', 'DR1TMOIS', 'DR1_320Z']

micro2_nutrient_list = ['DR2TATOC', 'DR2TRET', 'DR2TVARA', 'DR2TACAR', 
                        'DR2TBCAR', 'DR2TCRYP', 'DR2TLYCO', 'DR2TLZ',
                        'DR2TVB1', 'DR2TVB2', 'DR2TNIAC', 'DR2TVB6',
                        'DR2TFOLA', 'DR2TFA', 'DR2TFF', 'DR2TFDFE',
                        'DR2TCHL', 'DR2TVB12', 'DR2TVC', 'DR2TVD',
                        'DR2TVK', 'DR2TCALC', 'DR2TPHOS', 'DR2TMAGN',
                        'DR2TIRON', 'DR2TZINC', 'DR2TCOPP', 'DR2TSODI',
                        'DR2TPOTA', 'DR2TSELE', 'DR2TMOIS', 'DR2_320Z']

In [273]:
nhanes_diet_df['vit_e_avg'] = nhanes_diet_df[['DR1TATOC', 'DR2TATOC']].mean(axis=1)

In [279]:
nhanes_diet_df['retinol_avg'] = nhanes_diet_df[['DR1TRET', 'DR2TRET']].mean(axis=1)

In [280]:
nhanes_diet_df['vit_a_avg'] = nhanes_diet_df[['DR1TVARA', 'DR2TVARA']].mean(axis=1)

In [281]:
nhanes_diet_df['alpha_c_avg'] = nhanes_diet_df[['DR1TACAR', 'DR2TACAR']].mean(axis=1)

In [282]:
nhanes_diet_df['beta_c_avg'] = nhanes_diet_df[['DR1TBCAR', 'DR2TBCAR']].mean(axis=1)

In [283]:
nhanes_diet_df['beta_cry_avg'] = nhanes_diet_df[['DR1TCRYP', 'DR2TCRYP']].mean(axis=1)

In [284]:
nhanes_diet_df['lyco_avg'] = nhanes_diet_df[['DR1TLYCO', 'DR2TLYCO']].mean(axis=1)

In [285]:
nhanes_diet_df['lutein_avg'] = nhanes_diet_df[['DR1TLZ', 'DR2TLZ']].mean(axis=1)

In [286]:
nhanes_diet_df['vit_b1_avg'] = nhanes_diet_df[['DR1TVB1', 'DR2TVB1']].mean(axis=1)

In [287]:
nhanes_diet_df['vit_b2_avg'] = nhanes_diet_df[['DR1TVB2', 'DR2TVB2']].mean(axis=1)

In [288]:
nhanes_diet_df['niac_avg'] = nhanes_diet_df[['DR1TNIAC', 'DR2TNIAC']].mean(axis=1)

In [289]:
nhanes_diet_df['vit_b6_avg'] = nhanes_diet_df[['DR1TVB6', 'DR2TVB6']].mean(axis=1)

In [290]:
nhanes_diet_df['folate_avg'] = nhanes_diet_df[['DR1TFOLA', 'DR2TFOLA']].mean(axis=1)

In [291]:
nhanes_diet_df['folic_acid_avg'] = nhanes_diet_df[['DR1TFA', 'DR2TFA']].mean(axis=1)

In [292]:
nhanes_diet_df['food_fol_avg'] = nhanes_diet_df[['DR1TFF', 'DR2TFF']].mean(axis=1)

In [293]:
nhanes_diet_df['folate_dfe_avg'] = nhanes_diet_df[['DR1TFDFE', 'DR2TFDFE']].mean(axis=1)

In [294]:
nhanes_diet_df['chol_avg'] = nhanes_diet_df[['DR1TCHL', 'DR2TCHL']].mean(axis=1)

In [295]:
nhanes_diet_df['vit_b12_avg'] = nhanes_diet_df[['DR1TVB12', 'DR2TVB12']].mean(axis=1)

In [296]:
nhanes_diet_df['vit_c_avg'] = nhanes_diet_df[['DR1TVC', 'DR2TVC']].mean(axis=1)

In [297]:
nhanes_diet_df['vit_d_avg'] = nhanes_diet_df[['DR1TVD', 'DR2TVD']].mean(axis=1)

In [298]:
nhanes_diet_df['vit_k_avg'] = nhanes_diet_df[['DR1TVK', 'DR2TVK']].mean(axis=1)

In [300]:
nhanes_diet_df['calc_avg'] = nhanes_diet_df[['DR1TCALC', 'DR2TCALC']].mean(axis=1)

In [301]:
nhanes_diet_df['phos_avg'] = nhanes_diet_df[['DR1TPHOS', 'DR2TPHOS']].mean(axis=1)

In [ ]:
nhanes_diet_df['mag_avg'] = nhanes_diet_df[['DR1TMAGN', 'DR2TVMAGN']].mean(axis=1)

In [303]:
nhanes_diet_df['fe_avg'] = nhanes_diet_df[['DR1TIRON', 'DR2TIRON']].mean(axis=1)

In [304]:
nhanes_diet_df['zn_avg'] = nhanes_diet_df[['DR1TZINC', 'DR2TZINC']].mean(axis=1)

In [305]:
nhanes_diet_df['cu_avg'] = nhanes_diet_df[['DR1TCOPP', 'DR2TCOPP']].mean(axis=1)

In [306]:
nhanes_diet_df['na_avg'] = nhanes_diet_df[['DR1TSODI', 'DR2TSODI']].mean(axis=1)

In [307]:
nhanes_diet_df['k_avg'] = nhanes_diet_df[['DR1TPOTA', 'DR2TPOTA']].mean(axis=1)

In [308]:
nhanes_diet_df['se_avg'] = nhanes_diet_df[['DR1TSELE', 'DR2TSELE']].mean(axis=1)

In [309]:
nhanes_diet_df['mois_avg'] = nhanes_diet_df[['DR1TMOIS', 'DR2TMOIS']].mean(axis=1)

In [310]:
nhanes_diet_df['h20_avg'] = nhanes_diet_df[['DR1_320Z', 'DR2_320Z']].mean(axis=1)

In [311]:
nhanes_diet_df.head()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC,LBXHSCRP,WTDRD1_x,WTDR2D_x,DRDINT_x,DBQ095Z,DBD100,DRQSPREP,DR1STY,DRQSDIET,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TMOIS,DR1_320Z,DRD360,WTDRD1_y,WTDR2D_y,DRDINT_y,DR2STY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TMOIS,DR2_320Z,WTDRD1_x,WTDR2D_x,DS1DS,DS1AN,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TVC,DS1TVD,DS1TCALC,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE,WTDRD1_y,WTDR2D_y,DS2DS,DS2AN,DS2TVB1,DS2TVB2,DS2TNIAC,DS2TVB6,DS2TVB12,DS2TFA,DS2TFDFE,DS2TVC,DS2TVD,DS2TCALC,DS2TMAGN,DS2TIRON,DS2TZINC,DS2TCOPP,DS2TSODI,DS2TPOTA,DS2TSELE,diet_avg,kcal_avg,pro_avg,carb_avg,sugar_avg,fiber_avg,fat_avg,sfat_avg,mfat_avg,pfat_avg,chol_avg,vit_e_avg,retinol_avg,vit_a_avg,alpha_c_avg,beta_c_avg,beta_cry_avg,lyco_avg,lutein_avg,vit_b1_avg,vit_b2_avg,niac_avg,vit_b6_avg,folate_avg,folic_acid_avg,food_fol_avg,folate_dfe_avg,vit_b12_avg,vit_c_avg,vit_d_avg,vit_k_avg,calc_avg,phos_avg,mag_avg,fe_avg,zn_avg,cu_avg,na_avg,k_avg,se_avg,mois_avg,h20_avg
0,83732.0,1,62.0,134671.370419,135629.507405,4.39,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,94.8,184.5,27.8,35.9,101.1,22.8,212.0,225.0,2.0,230.0,205.0,260.0,20.0,2.0,2.0,9.0,9.0,9.0,2.0,1.0,1.0,2.0,9.0,122.666667,65.333333,9.8,23.9,8.2,63.5,4.0,0.5,2.3,0.8,6.2,0.4,4.93,15.2,44.7,90.8,30.8,13.9,181.0,8.3,4.6,52.0,21.0,25.0,13.0,25.0,9.8,172.0,203.0,100.0,0.90,94.0,70.0,4.30,110.0,136.0,272.0,4.7,1.518,0.5,7.5,158.0,4.2,46.0,173.0,0.6,92670.699919,69945.934107,2.0,1.0,3.0,3.0,1.0,2.0,1781.0,76.03,193.29,42.31,23.6,79.24,23.430,31.897,18.528,138.0,10.59,230.0,307.0,336.0,624.0,282.0,1206.0,1120.0,2.344,1.949,20.401,2.279,450.0,163.0,286.0,564.0,290.7,1.89,110.5,2.6,139.0,623.0,1052.0,255.0,16.01,8.73,1.095,5298.0,2641.0,113.6,360.0,3028.29,960.0,1.0,92670.699919,69945.934107,2.0,1.0,2744.0,133.60,281.64,118.40,15.9,121.59,40.420,42.428,26.813,635.0,8.62,494.0,996.0,91.0,5970.0,42.0,1715.0,8474.0,2.027,3.286,35.661,3.283,395.0,160.0,235.0,507.0,540.4,7.01,64.6,9.6,533.4,1433.0,1880.0,334.0,15.67,15.09,1.265,4431.0,3774.0,167.8,3694.92,1440.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,4.2,11.0,0.5,30.0,290.0,19.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,2.1,11.0,0.5,15.0,185.0,19.0,386.5,2262.5,104.815,237.465,80.355,19.75,100.415,31.9250,37.1625,22.6705,415.55,318.4745,362.0,651.5,213.5,3297.0,162.0,1460.5,4797.0,2.1855,2.6175,28.0310,2.7810,422.5,161.5,260.5,535.5,4.450,87.55,6.10,336.20,1028.0,1466.0,3361.605,15.840,11.910,1.1800,4864.5,3207.5,140.70,3361.605,1200.0
1,83733.0,1,53.0,24328.560239,25282.425927,1.32,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,90.4,171.4,30.8,33.2,107.9,27.3,193.0,19

In [ ]:
suppl_avg = ['supp_b1_avg', 'supp_b2_avg', 'supp_nia_avg', 'supp_b6_avg',
             'supp_b12_avg', 'supp_folate_avg', 'supp_folfde_avg', 
             'supp_c_avg', 'supp_d_avg', 'supp_ca_avg', 'supp_mg_avg', 
             'supp_fe_avg', 'supp_zn_avg', 'supp_cu_avg', 'supp_na_avg', 
             'supp_potk_avg', 'supp_se_avg']

supp1_nutrient_list = ['DS1TVB1', 'DS1TVB2', 'DS1TNIAC', 'DS1TVB6',
                       'DS1TVB12', 'DS1TFA', 'DS1TFDFE', 'DS1TVC',
                       'DS1TVD', 'DS1TCALC', 'DS1TMAGN', 'DS1TIRON',
                       'DS1TZINC', 'DS1TCOPP', 'DS1TSODI', 'DS1TPOTA',
                       'DS1TSELE']
    
supp2_nutrient_list = ['DS2TVB1', 'DS2TVB2', 'DS2TNIAC', 'DS2TVB6',
                       'DS2TVB12', 'DS2TFA', 'DS2TFDFE', 'DS2TVC',
                       'DS2TVD', 'DS2TCALC', 'DS2TMAGN', 'DS2TIRON',
                       'DS2TZINC', 'DS2TCOPP', 'DS2TSODI', 'DS2TPOTA',
                       'DS2TSELE']   

In [312]:
nhanes_diet_df['supp_b1_avg'] = nhanes_diet_df[['DS1TVB1', 'DS2TVB1']].mean(axis=1)

In [313]:
nhanes_diet_df['supp_b2_avg'] = nhanes_diet_df[['DS1TVB2', 'DS2TVB2']].mean(axis=1)

In [314]:
nhanes_diet_df['supp_nia_avg'] = nhanes_diet_df[['DS1TNIAC', 'DS2TNIAC']].mean(axis=1)

In [315]:
nhanes_diet_df['supp_b6_avg'] = nhanes_diet_df[['DS1TVB6', 'DS2TVB6']].mean(axis=1)

In [316]:
nhanes_diet_df['supp_b12_avg'] = nhanes_diet_df[['DS1TVB12', 'DS2TVB12']].mean(axis=1)

In [317]:
nhanes_diet_df['supp_folate_avg'] = nhanes_diet_df[['DS1TFA', 'DS2TFA']].mean(axis=1)

In [318]:
nhanes_diet_df['supp_folfde_avg'] = nhanes_diet_df[['DS1TFDFE', 'DS2TFDFE']].mean(axis=1)

In [320]:
nhanes_diet_df['supp_c_avg'] = nhanes_diet_df[['DS1TVC', 'DS2TVC']].mean(axis=1)

In [321]:
nhanes_diet_df['supp_d_avg'] = nhanes_diet_df[['DS1TVD', 'DS2TVD']].mean(axis=1)

In [322]:
nhanes_diet_df['supp_ca_avg'] = nhanes_diet_df[['DS1TCALC', 'DS2TCALC']].mean(axis=1)

In [323]:
nhanes_diet_df['supp_mg_avg'] = nhanes_diet_df[['DS1TMAGN', 'DS2TMAGN']].mean(axis=1)

In [324]:
nhanes_diet_df['supp_fe_avg'] = nhanes_diet_df[['DS1TIRON', 'DS2TIRON']].mean(axis=1)

In [325]:
nhanes_diet_df['supp_ze_avg'] = nhanes_diet_df[['DS1TZINC', 'DS2TZINC']].mean(axis=1)

In [326]:
nhanes_diet_df['supp_cu_avg'] = nhanes_diet_df[['DS1TCOPP', 'DS2TCOPP']].mean(axis=1)

In [327]:
nhanes_diet_df['supp_na_avg'] = nhanes_diet_df[['DS1TSODI', 'DS2TSODI']].mean(axis=1)

In [328]:
nhanes_diet_df['supp_potk_avg'] = nhanes_diet_df[['DS1TPOTA', 'DS2TPOTA']].mean(axis=1)

In [329]:
nhanes_diet_df['supp_se_avg'] = nhanes_diet_df[['DS1TSELE', 'DS2TSELE']].mean(axis=1)

In [331]:
nhanes_diet_df.head()

,SEQN,RIAGENDR,RIDAGEYR,WTINT2YR,WTMEC2YR,INDFMPIR,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ300A,MCQ365C,MCQ365D,MCQ370A,MCQ370C,MCQ370D,BMDSTATS,BMXWT,BMXHT,BMXBMI,BMXARMC,BMXWAIST,BMDAVSAD,WHD020,WHD050,WHQ225,WHD110,WHD120,WHD140,WHQ150,WHQ190,BPQ020,BPQ030,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D,bp_sys_avg,bp_dis_avg,LBXWBCSI,LBXLYPCT,LBXMOPCT,LBXNEPCT,LBXEOPCT,LBXBAPCT,LBDLYMNO,LBDMONO,LBDNENO,LBDEONO,LBXRBCSI,LBXHGB,LBXHCT,LBXMCVSI,LBXMCHSI,LBXRDW,LBXPLTSI,LBXMPSI,LBXSAL,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSBU,LBXSC3SI,LBXSCA,LBXSCH,LBXSCK,LBXSCLSI,LBXSCR,LBXSGL,LBXSIR,LBXSKSI,LBXSLDSI,LBXSNASI,LBXSOSSI,LBXSPH,LBDSPHSI,LBXSTB,LBXSTP,LBXSTR,LBXSUA,LBDHDD,LBXTC,LBXHSCRP,WTDRD1_x,WTDR2D_x,DRDINT_x,DBQ095Z,DBD100,DRQSPREP,DR1STY,DRQSDIET,DR1TKCAL,DR1TPROT,DR1TCARB,DR1TSUGR,DR1TFIBE,DR1TTFAT,DR1TSFAT,DR1TMFAT,DR1TPFAT,DR1TCHOL,DR1TATOC,DR1TRET,DR1TVARA,DR1TACAR,DR1TBCAR,DR1TCRYP,DR1TLYCO,DR1TLZ,DR1TVB1,DR1TVB2,DR1TNIAC,DR1TVB6,DR1TFOLA,DR1TFA,DR1TFF,DR1TFDFE,DR1TCHL,DR1TVB12,DR1TVC,DR1TVD,DR1TVK,DR1TCALC,DR1TPHOS,DR1TMAGN,DR1TIRON,DR1TZINC,DR1TCOPP,DR1TSODI,DR1TPOTA,DR1TSELE,DR1TCAFF,DR1TMOIS,DR1_320Z,DRD360,WTDRD1_y,WTDR2D_y,DRDINT_y,DR2STY,DR2TKCAL,DR2TPROT,DR2TCARB,DR2TSUGR,DR2TFIBE,DR2TTFAT,DR2TSFAT,DR2TMFAT,DR2TPFAT,DR2TCHOL,DR2TATOC,DR2TRET,DR2TVARA,DR2TACAR,DR2TBCAR,DR2TCRYP,DR2TLYCO,DR2TLZ,DR2TVB1,DR2TVB2,DR2TNIAC,DR2TVB6,DR2TFOLA,DR2TFA,DR2TFF,DR2TFDFE,DR2TCHL,DR2TVB12,DR2TVC,DR2TVD,DR2TVK,DR2TCALC,DR2TPHOS,DR2TMAGN,DR2TIRON,DR2TZINC,DR2TCOPP,DR2TSODI,DR2TPOTA,DR2TSELE,DR2TMOIS,DR2_320Z,WTDRD1_x,WTDR2D_x,DS1DS,DS1AN,DS1TVB1,DS1TVB2,DS1TNIAC,DS1TVB6,DS1TVB12,DS1TFA,DS1TFDFE,DS1TVC,DS1TVD,DS1TCALC,DS1TMAGN,DS1TIRON,DS1TZINC,DS1TCOPP,DS1TSODI,DS1TPOTA,DS1TSELE,WTDRD1_y,WTDR2D_y,DS2DS,DS2AN,DS2TVB1,DS2TVB2,DS2TNIAC,DS2TVB6,DS2TVB12,DS2TFA,DS2TFDFE,DS2TVC,DS2TVD,DS2TCALC,DS2TMAGN,DS2TIRON,DS2TZINC,DS2TCOPP,DS2TSODI,DS2TPOTA,DS2TSELE,diet_avg,kcal_avg,pro_avg,carb_avg,sugar_avg,fiber_avg,fat_avg,sfat_avg,mfat_avg,pfat_avg,chol_avg,vit_e_avg,retinol_avg,vit_a_avg,alpha_c_avg,beta_c_avg,beta_cry_avg,lyco_avg,lutein_avg,vit_b1_avg,vit_b2_avg,niac_avg,vit_b6_avg,folate_avg,folic_acid_avg,food_fol_avg,folate_dfe_avg,vit_b12_avg,vit_c_avg,vit_d_avg,vit_k_avg,calc_avg,phos_avg,mag_avg,fe_avg,zn_avg,cu_avg,na_avg,k_avg,se_avg,mois_avg,h20_avg,supp_b1_avg,supp_b2_avg,supp_nia_avg,supp_b6_avg,supp_b12_avg,supp_folate_avg,supp_folfde_avg,supp_c_avg,supp_d_avg,supp_ca_avg,supp_mg_avg,supp_fe_avg,supp_ze_avg,supp_cu_avg,supp_na_avg,supp_potk_avg,supp_se_avg
0,83732.0,1,62.0,134671.370419,135629.507405,4.39,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,94.8,184.5,27.8,35.9,101.1,22.8,212.0,225.0,2.0,230.0,205.0,260.0,20.0,2.0,2.0,9.0,9.0,9.0,2.0,1.0,1.0,2.0,9.0,122.666667,65.333333,9.8,23.9,8.2,63.5,4.0,0.5,2.3,0.8,6.2,0.4,4.93,15.2,44.7,90.8,30.8,13.9,181.0,8.3,4.6,52.0,21.0,25.0,13.0,25.0,9.8,172.0,203.0,100.0,0.90,94.0,70.0,4.30,110.0,136.0,272.0,4.7,1.518,0.5,7.5,158.0,4.2,46.0,173.0,0.6,92670.699919,69945.934107,2.0,1.0,3.0,3.0,1.0,2.0,1781.0,76.03,193.29,42.31,23.6,79.24,23.430,31.897,18.528,138.0,10.59,230.0,307.0,336.0,624.0,282.0,1206.0,1120.0,2.344,1.949,20.401,2.279,450.0,163.0,286.0,564.0,290.7,1.89,110.5,2.6,139.0,623.0,1052.0,255.0,16.01,8.73,1.095,5298.0,2641.0,113.6,360.0,3028.29,960.0,1.0,92670.699919,69945.934107,2.0,1.0,2744.0,133.60,281.64,118.40,15.9,121.59,40.420,42.428,26.813,635.0,8.62,494.0,996.0,91.0,5970.0,42.0,1715.0,8474.0,2.027,3.286,35.661,3.283,395.0,160.0,235.0,507.0,540.4,7.01,64.6,9.6,533.4,1433.0,1880.0,334.0,15.67,15.09,1.265,4431.0,3774.0,167.8,3694.92,1440.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,4.2,11.0,0.5,30.0,290.0,19.0,92670.699919,69945.934107,1.0,2.0,1.50,1.7,20.0,3.0,25.0,400.0,680.0,60.0,12.5,220.0,50.0,2.1,11.0,0.5,15.0,185.0,19.0,386.5,2262.5,104.815,237.465,80.355,19.75,100.415,31.9250,37.1625,22.6705,415.55,318.4745,362.0,651.5,213.5,3297.0,162.0,1460.5,4797.0,2.1855,2.6175,28.0310,2.7810,422.5,161.5,260.5,535.5,4.450,87.55,6.10,336.20

In [332]:
nhanes_diet_df.isnull().sum()

SEQN                  0
RIAGENDR              0
RIDAGEYR            379
WTINT2YR              0
WTMEC2YR              0
INDFMPIR           1025
MCQ160B               0
MCQ160C               0
MCQ160D               0
MCQ160E               0
MCQ300A               0
MCQ365C               0
MCQ365D               0
MCQ370A               0
MCQ370C               0
MCQ370D               0
BMDSTATS              0
BMXWT                99
BMXHT               775
BMXBMI              788
BMXARMC             568
BMXWAIST           1231
BMDAVSAD           2561
WHD020                0
WHD050                0
WHQ225                0
WHD110                0
WHD120                0
WHD140                0
WHQ150                0
WHQ190                0
BPQ020                0
BPQ030                0
BPQ040A               0
BPQ050A               0
BPQ080                0
BPQ060                0
BPQ070                0
BPQ090D               0
BPQ100D               0
bp_sys_avg         2181
bp_dis_avg      

In [ ]:
# fill in missing 'BMXWAIST', 'BMDAVSAD' with relationship of wt
# fill missing wt with avg for gender/age
# calculate missing BMI; if missing height, use avg height for gender/age

In [ ]:
# Dropping cases where birth year is not known
data2 =data[(data['BIRTHYR']<=2009) & (data['BIRTHYR'].notnull())]

# Calculating time-to-event
data2['Longevity']= data2['MORTDODY'] - data2['BIRTHYR']

# Creating a 'status' variable where 1 means dead and 0 means alive as of 2009
data2['dead']=2-data2['MORTSTAT']

# For people who were alive as of 2009, calculating their longevity as their age in 2009
data2.loc[data2['dead']==0,'Longevity'] = 2009 -data2.loc[data2['dead']==0,'BIRTHYR'] 

# Creating a binary variable where 1 = female and 0 = male
data2['female'] = (data2['SEX'] == 2).astype(int)



g = sns.FacetGrid(data2, row='female', col='dead',margin_titles=True)
g.map(plt.hist, 'Longevity', color='steelblue', lw=0)
plt.show()

In [ ]:
data2.head()

In [ ]:
data2.shape